In [1]:
import easyocr
import os
import numpy as np
import cv2
import pandas as pd
import time
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import rotate
from deskew import determine_skew
from textblob import TextBlob

In [2]:
reader = easyocr.Reader(['en','hi'])

In [18]:
def create_folders(folder_names, target_directory):
    # Ensure the target directory exists
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
    
    # Iterate through the list of folder names and create each one
    for folder_name in folder_names:
        folder_path = os.path.join(target_directory, folder_name)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Created folder: {folder_path}")
        else:
            print(f"Folder already exists: {folder_path}")

# Example usage
folder_names = ['20230106111705174401', '20230202173623542302', '20230202174013331804', '2023020917573151915 cir', 
                '20230216134112536906', '20230314175534332610', '20230327132153939511 cr', 'output_20230217163427693307',
                'output_2023031412003784278', 'output_2023031412023485669']

target_directory = r"C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed"

create_folders(folder_names, target_directory)

Created folder: C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed\20230106111705174401
Created folder: C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed\20230202173623542302
Created folder: C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed\20230202174013331804
Created folder: C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed\2023020917573151915 cir
Created folder: C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed\20230216134112536906
Created folder: C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed\20230314175534332610
Created folder: C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed\20230327132153939511 cr
Created folder: C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed\output_20230217163427693307
Created fol

In [19]:
image = io.imread(r"C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_special\20230106111705174401\20230106111705174401-page-00001.jpg")
grayscale = rgb2gray(image)
angle = determine_skew(grayscale)
rotated = rotate(image, angle, resize=True) * 255
io.imsave(r'Docs_to_jpg_deskewed\20230106111705174401\20230106111705174401-page-00001.jpg', rotated.astype(np.uint8))

In [20]:
# Specify the directory containing the folders
directory_path = r"C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_special"

# List all folders in the directory
folders = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]
# Process each folder
for folder in folders:
    folder_path = os.path.join(directory_path, folder)
    # List all JPG files in the folder
    jpg_files = [file for file in os.listdir(folder_path) if file.lower().endswith('.jpg')]
    for file in jpg_files:
        image = io.imread(os.path.join(folder_path, file))
        grayscale = rgb2gray(image)
        angle = determine_skew(grayscale)
        rotated = rotate(image, angle, resize=True) * 255
        new_path = f'Docs_to_jpg_deskewed\{folder}\{file}'
        io.imsave(new_path, rotated.astype(np.uint8))

# This cell took 309 seconds to run.

In [3]:
def extract_text_from_image(image_path):
    try:
    
        # Initialize the EasyOCR reader
        reader = easyocr.Reader(['en','hi'])  # 'hi' is the language code for Hindi
        
        # Read the image
        result = reader.readtext(image_path)
        
        # Load the image using OpenCV to get dimensions
        # image = cv2.imread(image_path)
        # image_height, image_width = image.shape[:2]
        
        # # Create a blank image to visualize the layout (optional)
        # blank_image = np.ones((image_height, image_width, 3), np.uint8) * 255
        
        # Initialize a list to hold the formatted lines of text
        formatted_lines = []
        
        # Process the OCR results
        for (bbox, text, prob) in result:
            # Get bounding box coordinates
            top_left, top_right, bottom_right, bottom_left = bbox
            x_min = int(min(top_left[0], bottom_left[0]))
            x_max = int(max(top_right[0], bottom_right[0]))
            y_min = int(min(top_left[1], top_right[1]))
            y_max = int(max(bottom_left[1], bottom_right[1]))
            
            # Draw the bounding box on the blank image (optional)
            # cv2.rectangle(blank_image, (x_min, y_min), (x_max, y_max), (0, 0, 0), 1)
            
            # Add the text to the corresponding position in the formatted lines
            formatted_lines.append((y_min, x_min, text))
        
        # Sort the lines based on y_min to maintain top-to-bottom order
        formatted_lines.sort()
        
        # Group text by lines, inserting newlines for paragraph changes
        output_lines = []
        current_line = []
        current_y = formatted_lines[0][0]
        
        # Threshold to detect paragraph changes, this might need adjustment based on document
        paragraph_threshold = 30
        
        for y_min, x_min, text in formatted_lines:
            if abs(y_min - current_y) <= paragraph_threshold:
                current_line.append((x_min, text))
            else:
                # Sort the current line by x_min
                current_line.sort()
                line_text = ' '.join([txt for _, txt in current_line])
                output_lines.append(line_text)
                
                # Insert a newline to indicate a paragraph change
                output_lines.append("\n")
                
                current_line = [(x_min, text)]
                current_y = y_min
        
        # Add the last line
        if current_line:
            current_line.sort()
            line_text = ' '.join([txt for _, txt in current_line])
            output_lines.append(line_text)
        
        # Remove unnecessary newlines
        formatted_text = '\n'.join(line for line in output_lines if line.strip() != "\n")
        
        # Save the blank image with bounding boxes (optional)
        # cv2.imwrite('output_with_bounding_boxes.jpg', blank_image)
        
        return formatted_text
        
    except Exception as e:
        print(f"Error processing file {image_path}: {e}")
        return ""

In [21]:
start_time = time.time()

# Specify the directory containing the folders
directory_path = r"C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_deskewed"

# List all folders in the directory
folders = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]

# Initialize an empty list to store folder names and their respective texts
folder_texts = []
count = 1
# Process each folder
for folder in folders:
    folder_path = os.path.join(directory_path, folder)
    # List all JPG files in the folder
    jpg_files = [file for file in os.listdir(folder_path) if file.lower().endswith('.jpg')]
    
    # Extract text from each JPG file and concatenate them into a single string
    extracted_texts = [extract_text_from_image(os.path.join(folder_path, file)) for file in jpg_files]
    combined_text = ' '.join(extracted_texts)
    
    # Append the folder name and the combined text to the list
    folder_texts.append((folder, combined_text))
    print(f"{count*10}% done")
    count += 1

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Time taken to run the program: {elapsed_time:.2f} seconds")

10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done
100% done
Time taken to run the program: 970.05 seconds


In [4]:
print(extract_text_from_image(r"C:\Users\ARITRA\Documents\Notebooks\ICG Stuff\UP-Police-Project\Docs_to_jpg_special\20230106111705174401\20230106111705174401-page-00001.jpg"))

देवेन्द्र सिंह चौहान, डीजी  परिपत्र संख्या- 0! २०२३


आई०पी०एस० पुलिस महानिदेशक,


उत्तर  प्रदेश |


उत्तर प्र< लॅर


पुलिस मुख्यालय, लखनऊ|


दिनांकःलखनऊःजनवरी०३ ,२०२३


विषयः शरद   ऋतु में   शीतलहर/घने कोहरे  की   रात्रि में   घुमक्कड़ बावरियों / कंच्छा


बनियानधारी गिरोहों के अपराधियों पर प्रभावी अंकुश लगाये जाने तथा अपराध


नियन्त्रण हेतु आवश्यक दिशा-निर्देश|


प्रिय  महोदय / महोदया,


आप अवगत हैं कि शरद ऋतु में शीतलहर घने कोहरे की रात्रि में विशेष रूप से सड़कों पर


आम जनमानस का आवागमन कम संख्या   में होता   है| इस मौसम  में घुमक्कड़ अपराधी बावरिया


अपराधी कच्छा बनियान गैंग द्वारा   सनसनीखेज अपराध कारित करने की सम्भावना रहती है| ऐसी


घटनाएंॅ प्रायः नगरों तथा कस्बों व कभी-्कभी ग्रामीण अंचल के बाहरी क्षेत्रों में घटित होती है|


डीजी परिपत्र संख्या-०१ २०२१, दिनांक 0१.0१.२०२१ आप सहमत होंगें कि इस प्रकार को घटना


डीजी-परिपत्र संख्या-२५ / २०११, दिनांक २७ .०6.२०१९ घटित होने पर   जिहॉ एक ओर जनमानस में


डीजी परिपत्र संख्या-६७ / २०१८, दिनांक २३. ११.२०१८ असुरक्षा का भय व्याप्त होता है वहीं 

In [22]:
folder_list = []
for i in range(10):
    print(folder_texts[i][0])
    folder_list.append(folder_texts[i][0])

20230106111705174401
20230202173623542302
20230202174013331804
2023020917573151915 cir
20230216134112536906
20230314175534332610
20230327132153939511 cr
output_20230217163427693307
output_2023031412003784278
output_2023031412023485669


In [3]:
folder_list = ['20230106111705174401', 
               '20230202173623542302',
               '20230202174013331804',
               '2023020917573151915 cir',
               '20230216134112536906',
               '20230314175534332610',
               '20230327132153939511 cr',
               'output_20230217163427693307', 
               'output_2023031412003784278',
               'output_2023031412023485669']

In [23]:
import re

def remove_special_characters(input_string):
    # Define the regular expression pattern to match unwanted characters
    pattern = r"[^a-zA-Z0-9\u0900-\u097F\[\]\(\)\{\}\/ ,.!|:\n;-]"
    # Remove unwanted characters
    cleaned_string = re.sub(pattern, '', input_string)
    # Remove unmatched brackets
    cleaned_string = remove_unmatched_brackets(cleaned_string)
    # Remove Hindi matras without adjoining Hindi letters
    cleaned_string = remove_unmatched_hindi_matras(cleaned_string)
    # Convert Hindi numerals to English numerals
    cleaned_string = convert_hindi_to_english_numerals(cleaned_string)
    # Replace sequences \nstr\n based on the given conditions
    cleaned_string = replace_specific_sequences(cleaned_string)
    # Remove characters after 'भवदीय' until '('
    cleaned_string = remove_characters_after_bhavdiya(cleaned_string)
    return cleaned_string
def remove_unmatched_brackets(input_string):
    stack = []
    result = []
    brackets = {'(': ')', '[': ']', '{': '}'}
    open_brackets = brackets.keys()
    close_brackets = brackets.values()
    
    for char in input_string:
        if char in open_brackets:
            stack.append(char)
            result.append(char)
        elif char in close_brackets:
            if stack and brackets[stack[-1]] == char:
                stack.pop()
                result.append(char)
            else:
                # Do not append unmatched closing bracket
                continue
        else:
            result.append(char)
    
    # Remove unmatched opening brackets
    unmatched_open_brackets = set(stack)
    final_result = []
    for char in result:
        if char in open_brackets and char in unmatched_open_brackets:
            unmatched_open_brackets.remove(char)
        else:
            final_result.append(char)
    
    return ''.join(final_result)

def remove_unmatched_hindi_matras(input_string):
    # Define the range for Hindi matras (vowel signs)
    hindi_matras = r'\u093E-\u094D\u0951-\u0957\u0962-\u0963'
    hindi_letters = r'\u0900-\u0914\u0915-\u0939\u0958-\u0961\u0966-\u096F'
    
    # Regular expression to find matras not preceded or followed by a Hindi letter
    pattern = rf'(?<![{hindi_letters}])[{hindi_matras}](?![{hindi_letters}])'
    
    # Remove such matras
    cleaned_string = re.sub(pattern, '', input_string)
    
    return cleaned_string

def convert_hindi_to_english_numerals(input_string):
    hindi_to_english = {
        '०': '0', '१': '1', '२': '2', '३': '3', '४': '4', 
        '५': '5', '६': '6', '७': '7', '८': '8', '९': '9'
    }
    return ''.join(hindi_to_english.get(char, char) for char in input_string)

def replace_specific_sequences(input_string):
    def replace_match(match):
        str_match = match.group(1)
        # Check if str_match contains 'विषय', is '०', has no spaces, or no numbers
        if 'विषय' in str_match:
            return match.group(0)  # Keep the whole match if it contains "विषय"
        if str_match == '0' or (not any(char.isdigit() for char in str_match) and ' ' not in str_match):
            return '\n'
        return match.group(0)  # Return the whole match if it doesn't meet the conditions
    
    # Regular expression to find sequences of the type \nstr\n
    pattern = re.compile(r'\n([^\n]+)\n')
    cleaned_string = pattern.sub(replace_match, input_string)
    
    return cleaned_string

def remove_characters_after_bhavdiya(input_string):
    # Regular expression to find 'भवदीय' followed by any characters until the next '('
    pattern = re.compile(r'(भवदीय[^(]*\()')
    
    # Replace the matched pattern with 'भवदीय('
    cleaned_string = pattern.sub('भवदीय (', input_string)
    
    return cleaned_string

# Example usage
example_string = "Hello! यह एक परीक्षण स्ट्रिंग है जिसमें विशेष अक्षर हैं: @#$%^&*()[]{})<>/.,|-\nHello123\n\nnoSpacesHere\n\nविषय\n\n०\n\ncontainsविषयtext\n१\nभवदीय abcdef( ghi"
cleaned_string = remove_special_characters(example_string)
print(cleaned_string)

Hello! यह एक परीक्षण स्ट्रिंग है जिसमें विशेष अक्षर हैं: )[]{}/.,|-
Hello123


विषय


containsविषयtext
1
भवदीय ( ghi


In [24]:
outputs = []
for i in range(10):
    outputs.append(remove_special_characters(folder_texts[i][1]))

In [49]:
print(outputs[9])

देवेन्द्र सिंह चौहान ,


डीजी परिपत्र सं0 - 09  /2723


आई0पी0एस0 पुलिस महानिदेशक


उत्तर प्रदेश


पुलिस मुख्यालय, गोमती नगर विस्तार,


उत्तर भ्रदरा लॅर


लखनऊ-226010


दिनांकःमार्च 02 2023


विषयः रिट सी॰ संख्या 29605/2022 बासु यादव बनाम भारत संघ व 4 अन्य में मा0 उच्च


न्यायालय इलाहाबाद द्वारा पारित आदेश दिनांकित 16.12 2022 के अनुपालन में पासपोर्ट


आवेदन पत्र के पुलिस सत्यापन हेतु दिशा-निर्देश |


प्रिय महोदया / महोदय,


पारपत्र (Passport) जारी करने हेतु प्रस्तुत किये गये आवेदन पत्रों के संदर्भ में आवेदकों


का पुलिस सत्यापन, सम्बन्धित थाना द्वारा किया जाता है आवेदक बासु यादव निवासी जनपद


आजमगढ़ द्वारा प्रस्तुत पासपोर्ट आवेदन पत्र पर स्थानीय पुलिस द्वारा पासपोर्ट न प्रदान किये जाने


की संस्तुति के विरुद्ध योजित सिविल मिस. रिट (सी) संख्याः29605/2022 बासु यादव बनाम


उ0प्र0 राज्य में मा0 उच्च न्यायालय द्वारा पारित आदेश दिनांकित 16 12 2022 में   पासपो्ट


आवेदन पत्र सत्यापन के सम्बन्ध में निम्नवत निर्देश दिये गये हैं-


Hlaving heard learned counscl for the pctitioner and Icarned Sta

In [ ]:
# Now we pass each element in 'outputs_updated' through ChatGPT with the following prompt - 

""" You are a text corrector,
I am going to provide you a text in Hindi and English. Clean up the text, by removing all random symbols, characters, etc which are clearly printing mistakes and do not make any sense.

Finally, output the corrected text.

DO NOT CHANGE anything else in the text.

Text - (we give the text, like output[0]) """

# For these 10 docs, I have directly used the ChatGPT web platform. In future, we will do it using ChatGPT API, after getting access to the API key.

In [5]:
#ChatGPT outputs

output1 = """
देवेन्द्र सिंह चौहान, डीजी परिपत्र संख्या- 01/ 2023

आईपीएस पुलिस महानिदेशक,

उत्तर प्रदेश

पुलिस मुख्यालय, लखनऊ

दिनांक: लखनऊ: जनवरी 03, 2023

विषय: शरद ऋतु में शीतलहर/घने कोहरे की रात्रि में घुमक्कड़ बावरियों/कच्छा बनियानधारी गिरोहों के अपराधियों पर प्रभावी अंकुश लगाये जाने तथा अपराध नियन्त्रण हेतु आवश्यक दिशा-निर्देश

प्रिय महोदय/महोदया,

आप अवगत हैं कि शरद ऋतु में शीतलहर घने कोहरे की रात्रि में विशेष रूप से सड़कों पर आम जनमानस का आवागमन कम संख्या में होता है। इस मौसम में घुमक्कड़ अपराधी बावरियां/कच्छा बनियान गैंग द्वारा सनसनीखेज अपराध कारित करने की सम्भावना रहती है। ऐसी घटनाएं प्रायः नगरों तथा कस्बों व कभी-कभी ग्रामीण अंचल के बाहरी क्षेत्रों में घटित होती हैं।

डीजी परिपत्र संख्या-01/2021, दिनांक 01.01.2021, आप सहमत होंगे कि इस प्रकार की घटना डीजी-परिपत्र संख्या-25/2011, दिनांक 27.06.2019 घटित होने पर जहां एक ओर जनमानस में डीजी परिपत्र संख्या-67/2018, दिनांक 23.11.2018 असुरक्षा का भय व्याप्त होता है, वहीं दूसरी ओर डीजी-परिपत्र संख्या-47/2016, दिनांक 03.08.2016 पुलिस के प्रति आक्रोश उत्पन्न होता है। इसलिए अत्यन्त आवश्यक है कि इस मौसम में गश्त व्यवस्था प्रभावी ढंग से सुनिश्चित करायी जाये जिससे जन-सामान्य में सुरक्षा की भावना बनी रहे तथा इस प्रकार की घटनाओं पर अंकुश लगाया जा सके।

इस प्रकार के रात्रि में घटित होने वाले अपराध पर प्रभावी अंकुश लगाये जाने हेतु निम्नांकित कार्यवाही अपेक्षित है:

शीतकाल में रात्रि के समय कोहरे के कारण दृश्यता कम होती है, इसका लाभ घुमक्कड़ बावरियां/कच्छा बनियानधारी अपराधियों के गैंग विचरण करते हैं तथा डकैती जैसी नकबजनी, एटीएम चोरी, शटर काटकर चोरी, बिजली तार चोरी तथा रोड होल्डअप जैसी दुःस्साहसिक सनसनीखेज घटनाएं कारित करते हैं, जिनमें प्रायः जनहानि भी हो जाती है। ऐसी वीभत्स घटनाएं घटित न होने पायें, इस हेतु कमिश्नरेट जनपद, परिक्षेत्र तथा जोन स्तर पर प्रभावी, व्यावहारिक कार्ययोजना तैयार कर समस्त पुलिस बल, गश्ती वाहन पर नियुक्त कर्मियों, राजपत्रित अधिकारियों की यथोचित ब्रीफिंग कर ली जाये। विगत में घटित घटनाओं की समीक्षा करते हुए जनपद/परिक्षेत्र व जोन स्तर पर हॉट-स्पॉट्स चिन्हित करते हुए सुदृढ़ पुलिस प्रबन्ध, योजनाबद्ध गश्त प्रणाली कार्यान्वित की जाये।

जनपद के ऐसे क्षेत्र जो इस प्रकार के अपराधों के लिए संवेदनशील सम्भावित हो सकते हों, ऐसे चिन्हित हॉटस्पॉट्स पर विशेष रूप से रात्रि 12:00 बजे से प्रातः 04:00 बजे के बीच नियमित गश्त करायी जाये। संवेदनशील राजमार्गों पर गश्त हेतु पर्याप्त संख्या में पेट्रोलिंग वाहन लगाये जायें। संवेदनशील स्थानों पर सुदृढ़ पुलिस पिकेट लगायी जाये। रात्रि गश्त हेतु थानों एवं चौकियों में पर्याप्त संख्या में पुलिस बल को नियुक्त किया जाये। गश्त चेकिंग के दौरान हूटर, सायरन, फ्लैश लाइट प्रभावी प्रयोग किया जाये।

प्रत्येक जनपद में रात्रि गश्त हेतु जोनल चेकिंग स्कीम व गश्त मिलान प्रणाली को सक्रिय कर लिया जाये। प्रतिदिन जनपद के प्रत्येक थाने में रात्रि में एक अधिकारी को यह जिम्मेदारी दी जाये कि वह गश्त व्यवस्था का मूल्यांकन/समीक्षा एवं चेकिंग सुनिश्चित करेंगे। ग्राम/मोहल्ला सुरक्षा समितियों को क्रियाशील बनाकर गश्त करायी जाये। रात्रि में गश्त करने वाले कर्मचारियों के पास डंडा, रॉड, सोटी इत्यादि उपकरण अवश्य हों तथा कभी भी अकेले गश्त पर कर्मचारियों को नहीं भेजा जाये। कम्यूनिकेशन प्लान का प्रभावी प्रयोग भी इस दिशा में लाभप्रद होगा। ग्रामों/मोहल्लों में डिजिटल वॉलिंटियर की संख्या बढ़ायी जाये ताकि सूचना का आदान-प्रदान त्वरित हो सके। यूपी-112 की गाड़ियों में लगे स्टॉफ को इस सम्बन्ध में विशेष रूप से ब्रीफ किया जाये तथा आवश्यकतानुसार इस दृष्टिकोण से उनका क्षेत्र पुनर्निर्धारित किया जाये।

घुमक्कड़ अपराधियों के ठहरने के ठिकानों के विषय में शीघ्रातिशीघ्र पूरे जनपद में छानबीन करायी जाये। विशेष रूप से शहर, कस्बों के बाहरी इलाकों, रेलवे लाइन एवं सड़कों के आस-पास लगे अस्थायी टेंटों पर चेकिंग करायी जाये, किन्तु यह भी सुनिश्चित करा लिया जाये कि किसी भी निर्दोष व्यक्ति को अनावश्यक परेशान न किया जाये। रेलवे स्टेशन तथा बस स्टेशन के आस-पास भी इस प्रकार की घटनाओं में संलिप्त अपराधियों का आवागमन व गतिविधियां रहती हैं। रेलवे स्टेशन के आस-पास ये प्रायः अपने अस्थायी ठिकाने बना लेते हैं तथा घटनाएं कारित करने के लिए गैंग के सदस्यों के साथ आवागमन हेतु ट्रेनों का प्रयोग करते हैं। रेलवे लाइनों के आस-पास की कालोनियों में भी प्रायः घुमक्कड़ अपराधियों द्वारा घटनाएं कारित की जाती हैं। अतः उक्त के दृष्टिगत जीआरपी के अधिकारियों के साथ समन्वय स्थापित कर गश्त चेकिंग की कार्यवाही प्रभावी रूप से की जाये। जीआरपी के भी समस्त कर्मचारियों व राजपत्रित अधिकारियों को इस सम्बन्ध में उपरोक्तानुसार योजनाबद्ध रूप से कार्यवाही हेतु यथोचित ब्रीफ किया जाये।

इस प्रकार के गिरोहों के सदस्य प्रायः घटना करने से पूर्व होटलों, ढाबों व शराब की दुकानों के आस-पास शराब का सेवन करते हैं। अतः रात्रि में शराब ठेकों, ढाबों इत्यादि के पास भी संदिग्धों की चेकिंग की जाये। ऐसे अपराधी कई बार घटनास्थल से 20-30 किलोमीटर की दूरी पर अपने वाहन खड़े कर जाते हैं, ताकि इस वाहन का प्रयोग वारदात करने के पश्चात् जनपद से भागने में किया जा सके। अतः ऐसे लावारिस वाहनों की चेकिंग अवश्य की जाये। घटना होने के पश्चात् जनपद की सीमा को सील करके ऐसे वाहनों को जनपद के बाहर न निकलने देने का प्रयास किया जाये। ढाबा एवं एकांत में खड़े ऐसे बड़े वाहनों की चेकिंग की जाये और देखा जाये कि वाहन में कोई सीक्रेट कम्पार्टमेन्ट/चेम्बर तो नहीं बनाये गये हैं। यदि बने हों तो उनमें देखा जाये कि अवैध वस्तु, असलहा आदि तो नहीं हैं।

समस्त जनपद, कमिश्नरेट, परिक्षेत्र व जोन स्तर पर विगत 10 वर्षों में इस प्रकार के अपराधों में सम्मिलित रहे अपराधियों को सूचीबद्ध करते हुए उनके सत्यापन की कार्यवाही हेतु प्रभावी अभियान प्रारम्भ किया जाये। सक्रिय संदिग्ध अपराधियों व उनके गैंग के विरुद्ध सख्त निरोधात्मक कार्यवाही सुनिश्चित की जाये। जनपद/परिक्षेत्र व जोन स्तर पर समन्वय स्थापित कर योजनाबद्ध रूप से यह कार्यवाही की जाये।

पूर्व में इस प्रकार की घटनाओं में पंजीकृत अभियोगों की विवेचनात्मक कार्यवाही की समीक्षा कर ली जाये। अनावरण को शेष समस्त प्रकरणों का ठोस कार्यवाही कर अनावरण कराया जाये। लम्बित विवेचनाओं में शेष कार्यवाही पूर्ण कराकर निस्तारण कराया जाये तथा वांछितों की गिरफ्तारी करायी जाये। इस प्रकार के गिरोहों एवं उनके सदस्यों की गैंगचार्ट तैयार कर उनके विरुद्ध गैंगस्टर एक्ट के तहत कार्यवाही करना भी सुनिश्चित किया जाये। इन अपराधियों के हिस्ट्रीशीट खोलने की कार्यवाही भी की जाये, ताकि इनकी निगरानी सतत् रूप से हो सके।

इस प्रकार के सनसनीखेज जघन्य अपराधों में संलिप्त रहे अपराधियों के विरुद्ध न्यायालय में विचाराधीन प्रकरणों में प्रभावी पैरवी सुनिश्चित की जाये। इनके जमानतदारों का पता लगाकर नियमानुसार सत्यापन कराया जाये तथा अभियुक्त अपराधियों की जमानत निस्तीकरण व विधिक कार्यवाही करायी जाये।

स्पेशल टॉस्क फोर्स (एसटीएफ) द्वारा अपनी फील्ड इकाइयों को उपरोक्तानुसार सक्रिय करते हुए घुमक्कड़ बावरियां/कच्छा बनियानधारी गैंग अपराधियों पर सतर्क चौकसी रखते हुए विशेष अभियान चलाकर यथावश्यक जनपदीय पुलिस से समन्वय स्थापित कर ठोस कार्यवाही सुनिश्चित की जाये।

उपरोक्त दिशा-निर्देश केवल सामान्य मार्ग-दर्शन के लिए हैं, इसके अतिरिक्त आप अपने जनपदों की अपराधिक एवं भौगोलिक परिस्थितियों के आधार पर अपराध नियन्त्रण की दिशा में अन्य आवश्यक कदम उठाने के लिए स्वतन्त्र हैं।

आप सभी से अपेक्षा है कि उपरोक्त बिन्दुओं का स्वयं गम्भीरता से अध्ययन कर लें एवं एक कार्यशाला के माध्यम से जनपद में नियुक्त सभी अधिकारियों व कर्मचारियों को इस निर्देश के सम्बन्ध में विस्तार से अवगत करा दें तथा इस सम्बन्ध में सतर्क कर दें कि वह अपने दायित्वों के निर्वहन में किसी भी प्रकार की लापरवाही व उदासीनता न बरतें।

उपरोक्त निर्देशों का अनुपालन कड़ाई से सुनिश्चित करें।

महोदय

देवेन्द्र सिंह चौहान

समस्त पुलिस आयुक्त, उत्तर प्रदेश

समस्त वरिष्ठ पुलिस अधीक्षक/पुलिस अधीक्षक,

प्रभारी जनपद उत्तर प्रदेश

प्रतिलिपि-निम्नांकित अधिकारियों को सूचनार्थ एवं आवश्यक कार्यवाही हेतु प्रेषित:

1. अपर पुलिस महानिदेशक, अपराध, उ.प्र. लखनऊ
2. अपर पुलिस महानिदेशक, रेलवेज/यूपी 112, उ.प्र. लखनऊ
3. अपर पुलिस महानिदेशक, एसटीएफ, उ.प्र. लखनऊ
4. समस्त जोनल अपर पुलिस महानिदेशक, उत्तर प्रदेश
5. पुलिस महानिरीक्षक अभिसूचना, उ.प्र. लखनऊ
6. समस्त परिक्षेत्रीय पुलिस महानिरीक्षक, पुलिस उपमहानिरीक्षक, उत्तर प्रदेश
"""

output2 = """
डीजी परिपत्र संख्या: 01/2023

देवेन्द्र सिंह चौहान

पुलिस महानिदेशक,

आई.पी.एस.

उत्तर प्रदेश

पुलिस मुख्यालय गोमती नगर विस्तार,

लखनऊ-226010

दिनांक: जनवरी 27, 2023

विषय: मा. संसद सदस्य एवं राज्य विधान मण्डल के प्रति शिष्टाचार / अनुमन्य प्रोटोकॉल एवं सौजन्य प्रदर्शन के अनुपालन के संबंध में मा. सदस्यों एवं अन्य जन प्रतिनिधियों के

प्रिय महोदय / महोदया,

आपका ध्यान मा. संसद सदस्य एवं राज्य विधान मण्डल के मा. सदस्यों एवं जनप्रतिनिधियों के प्रति शिष्टाचार / अनुमन्य प्रोटोकॉल एवं सौजन्य प्रदर्शन हेतु प्रचलित व्यवस्था की ओर आकृष्ट करना चाहूंगा। मा. संसद सदस्य एवं राज्य विधान मण्डल के मा. सदस्यों के प्रति शिष्टाचार / अनुमन्य प्रोटोकॉल के संबंध में उ.प्र. शासन एवं इस मुख्यालय से समय-समय पर पार्श्वांकित कॉलम में अंकित शासनादेश एवं निर्देश निर्गत किये गये हैं। यह संज्ञान में आया है कि इन निर्देशों के सम्यक अनुपालन में शिथिलता बरती जा रही है।

उक्त संबंध में समय-समय पर निर्गत आदेशों एवं निर्देशों का प्रत्येक थाना, सर्किल, जनपद, परिक्षेत्र, जोन एवं पुलिस कमिश्नरेट स्तर पर अनुपालन करते हुए गुणवत्तापूर्ण प्रदर्शन अपेक्षित है। स्थानीय स्तर पर इन महानुभावों के प्रति अनुमन्य प्रोटोकॉल एवं शिष्टाचार का समुचित प्रदर्शन न होने पर मा. संसद सदस्य एवं राज्य विधान मण्डल के मा. सदस्यों व अन्य जन प्रतिनिधियों द्वारा उ.प्र. शासन में एवं इस मुख्यालय में अधिकारियों से शिकायतें की जाती हैं। महानुभावों की शिकायत से जन सामान्य में पुलिस की छवि धूमिल होती है तथा वरिष्ठ अधिकारियों के समक्ष अरुचिकर स्थिति भी उत्पन्न होती है। इस संबंध में निम्न प्रकार कार्यवाही किये जाने की अपेक्षा की जाती है:

1-जनप्रतिनिधियों के फोन कॉल्स को रिकॉर्ड / वायरल किया जाना:
यह शिकायत प्राप्त हो रही है कि कतिपय पुलिस अधिकारियों एवं कर्मचारियों द्वारा मा. संसद सदस्य एवं राज्य विधान मण्डल के मा. सदस्यों के फोन से वार्ता के दौरान उनके फोन कॉल्स को रिकॉर्ड किया जाता है तथा वायरल भी कर दिया जाता है। यह प्रवृत्ति उचित नहीं है। विभाग के अधिकारियों कर्मचारियों के ऐसे क्रियाकलाप से मीडिया / सोशल मीडिया आदि के माध्यम से जनसामान्य में विभाग के प्रति अनावश्यक विवाद को बढ़ावा मिलता है एवं वरिष्ठ अधिकारियों को असहज स्थिति का सामना करना पड़ता है, जो आपत्तिजनक है। अतः सभी अधीनस्थ कार्मिकों को निर्देशित करें कि पुलिस अधिकारियों एवं कर्मचारियों द्वारा मा. संसद सदस्य एवं राज्य विधान मण्डल के मा. सदस्यों के फोन से वार्ता के दौरान उनके फोन कॉल्स को रिकॉर्ड / वायरल न किया जाये।

2-जनप्रतिनिधियों के प्रति शिष्टाचार प्रदर्शित किया जाये:
प्रत्येक थाना, सर्किल, जनपद, परिक्षेत्र, जोन एवं पुलिस कमिश्नरेट स्तर पर पुलिस अधिकारियों एवं कर्मचारियों द्वारा मा. संसद सदस्य एवं राज्य विधान मण्डल के मा. सदस्यों व अन्य जनप्रतिनिधियों के साथ गोष्ठी / बैठक, उनके भ्रमण के दौरान तथा फोन पर वार्ता करते समय उनके साथ प्रोटोकॉल के तहत शिष्टाचार / सम्मान प्रदर्शित किया जाना चाहिये। ऐसा न करने पर उनके द्वारा शिकायत की जाती है और विभाग को प्रतिकूल स्थिति का सामना करना पड़ता है जो चिंताजनक है। अतः प्रत्येक स्तर पर सभी पुलिस कार्मिकों द्वारा मा. संसद सदस्य एवं राज्य विधान मण्डल के मा. सदस्यों को अनुमन्य प्रोटोकॉल के अनुसार उचित शिष्टाचार के साथ सम्मान प्रदर्शित किया जाये।

3-समस्त पुलिस कर्मियों द्वारा अपनी वर्दी पर नाम पट्टिका लगाया जाये:
यह संज्ञान में आया है कि कतिपय पुलिस अधिकारियों एवं कर्मचारियों द्वारा वर्दी पर नाम पट्टिका नहीं धारण की जाती है। यह प्रचलित नियमों के अनुकूल नहीं है तथा यह प्रवृत्ति भी ठीक नहीं है। सभी थाना, सर्किल, जनपद, परिक्षेत्र, जोन एवं पुलिस कमिश्नरेट स्तर पर समस्त पुलिस अधिकारियों एवं कर्मचारियों द्वारा निर्धारित वर्दी के साथ नाम पट्टिका अवश्य धारण की जाये।

अतः आप सभी को निर्देशित किया जाता है कि उपरोक्त निर्देशों का अनुपालन करने हेतु अधीनस्थों को निर्देशित करते हुए कड़ाई से अनुपालन कराना सुनिश्चित किया जाये।

(देवेन्द्र सिंह चौहान)

1. समस्त पुलिस महानिदेशक, उ.प्र.
2. समस्त जोनल अपर पुलिस महानिदेशक, उ.प्र.
3. समस्त अपर पुलिस महानिदेशक, उ.प्र.
4. समस्त पुलिस आयुक्त, उ.प्र.
5. समस्त परिक्षेत्रीय पुलिस महानिरीक्षक / पुलिस उप महानिरीक्षक, उ.प्र.
6. समस्त वरिष्ठ पुलिस अधीक्षक / पुलिस अधीक्षक, प्रभारी जनपद, उ.प्र.
"""

output3 = """
देवेन्द्र सिंह चौहान,

डीजी परिपत्र सं0 - 05/2023

पुलिस महानिदेशक,

उत्तर प्रदेश

पुलिस मुख्यालय, गोमती नगर विस्तार,

लखनऊ-226010

दिनांक: जनवरी 30, 2023

विषय: मा. उच्च न्यायालय इलाहाबाद तथा खण्डपीठ लखनऊ में योजित पत्र याचिकाओं (Letter Petitions) पर तथ्यपरक, प्रासंगिक एवं विधिक आख्या प्रेषित किये जाने के सम्बन्ध में।

प्रिय महोदया / महोदय,

अवगत कराना है कि वर्तमान में मा. उच्च न्यायालय इलाहाबाद तथा खण्डपीठ लखनऊ में पत्र याचिकाएं (Letter Petitions) काफी संख्या में प्रस्तुत हो रही हैं। सरकार की ओर से प्रतिवाद करने वाले विधि अधिकारियों द्वारा ऐसी पत्र याचिकाओं पर अल्प समय में आख्या उपलब्ध कराने की अपेक्षा की जाती है। जिस कारण ऐसी आख्या सामान्यतया थानाध्यक्ष, चौकी प्रभारी, हल्का प्रभारी एवं अन्य उप निरीक्षक द्वारा सीधे प्रेषित की जा रही हैं।

मा. उच्च न्यायालय में सरकार की ओर से प्रतिवाद करने वाले विधि अधिकारियों द्वारा यह अवगत कराया गया है कि पत्र याचिकाओं (Letter Petitions) पर प्रेषित आख्या (Comments) में इस प्रकार के तथ्यों का समावेश किया जा रहा है, जो न तो विधि सम्मत होते हैं और न ही प्रासंगिक। जनपदों से प्राप्त ऐसी असंगत आख्या के मा. न्यायालय के समक्ष प्रस्तुत किये जाने पर मा. उच्च न्यायालय द्वारा विपरीत निष्कर्ष निकाले जाने की प्रबल सम्भावना होती है।

यहाँ पर यह भी उल्लेखनीय है कि सामान्य रूप से पत्र याचिका (Letter Petition) अत्यंत गंभीर विषय होने पर ही मा. न्यायालय द्वारा इन पर संज्ञान लिया जाता है। ऐसी दशा में प्रकरण से संबंधित तथ्यपरक एवं सुसंगत आख्या उपलब्ध न होने पर राज्य का पक्ष रखने वाले विधि अधिकारियों की स्थिति मा. न्यायालय के समक्ष असहज होती है तथा विपरीत आदेश पारित होने की सम्भावना बनी रहती है।

अतः आप सभी को निर्देशित किया जाता है कि पत्र याचिकाओं (Letter Petitions) पर मा. उच्च न्यायालय को प्रेषित की जाने वाली आख्या को पूर्ण सतर्कता के साथ तथा प्रकरण से संबंधित तथ्यपरक सूचनाओं का समावेश करते हुए तैयार किया जाए। पत्र याचिका में उल्लिखित तथ्यों का प्रतिवाद विधि सम्मत एवं तथ्यों के आधार पर किया जाए।

मा. उच्च न्यायालय को प्रेषित की जाने वाली आख्या को प्रत्येक पुलिस अधीक्षक के स्तर से परीक्षणोपरान्त विधिवत अनुमोदित एवं अग्रसारित कराया जायेगा। संबंधित अधिकारी अग्रसारण से पूर्व यह सुनिश्चित करेंगे कि प्रेषित की जाने वाली आख्या में अप्रासंगिक, अयुक्तियुक्त तथा अविधिक तथ्यों का समावेश कदापि न हो।

भवदीय

(देवेन्द्र सिंह चौहान)

पुलिस आयुक्त,
कमिश्नरेट लखनऊ, कानपुर, वाराणसी, गौतमबुद्धनगर, आगरा, गाजियाबाद, प्रयागराज

समस्त वरिष्ठ पुलिस अधीक्षक / पुलिस अधीक्षक,
प्रभारी जनपद रेलवेज, उत्तर प्रदेश

प्रतिलिपि: निम्नलिखित को सूचनार्थ एवं आवश्यक कार्यवाही हेतु:

1. समस्त पुलिस महानिदेशक, उ.प्र.
2. समस्त जोनल अपर पुलिस महानिदेशक, उ.प्र.
3. समस्त अपर पुलिस महानिदेशक, उ.प्र.
4. समस्त पुलिस महानिरीक्षक / उपमहानिरीक्षक, परिक्षेत्र, उ.प्र.
"""

output4 = """
डीजी परिपत्र सं0- 05/2023

देवेन्द्र सिंह चौहान पुलिस महानिदेशक

आई.पो.एरा0 उत्तर प्रदेश |

पुलिस मुख्यालय गोमती नगर विस्तार

लखनऊ-228010

दिनांकः फरवरी 08, 2023

विषयः उ0प0 पुलिस सोशल मीडिया पॉलिसी-2023

प्रिय महोदया / महादय,

सूचित करना है कि पुलिस द्वारा विगत कई वर्षों से सोशल मीडिया का सार्थक प्रयोग करके जनशिकायत का निस्तारण एवं सराहनीय कार्यों को सफलतापूर्वक किया जा रहा है। सोशल मीडिया पर डाली गई सामग्री पब्लिक प्लेटफॉर्म पर सबके लिए सुलभतासंयुक्त है अतएव विभागीय गरिमा के दृष्टिकोण से पुलिस अधिकारियों / कर्मचारियों द्वारा व्यक्तिगत रूप से सोशल मीडिया के प्रयोगार्थ पुलिस महानिदेशक उत्तर प्रदेश द्वारा दिनांक 28 मार्च 2017 को परिपत्र संख्या 08 / 2017 एवं दिनांक 04 अक्टूबर 2018 को परिपत्र संख्या 53 / 2018 के माध्यम से पुलिस कार्मिकों के लिए विस्तृत सोशल मीडिया पॉलिसी निर्गत की गयी है।

विगत कई दिनों से ऐसे दृष्टान्त सामने आ रहे हैं जहाँ पर पुलिस कार्मिकों द्वारा सोशल मीडिया पॉलिसी एवं उ0प्र0 सरकारी कर्मचारी आचरण नियमावली का उल्लंघन करते हुए कार्य करते समय सोशल मीडिया के प्रयोग एवं बावर्दी अशोभनीय रूप से वीडियो बनाकर उन्हें सोशल मीडिया के विभिन्न प्लेटफार्म पर अपलोड किया गया है, जिससे पुलिस विभाग की छवि धूमिल हुई है। इस प्रकार के कई प्रकरणों में संबंधित पुलिस कार्मिकों के विरुद्ध सख्त कार्यवाही भी की गयी है। भविष्य में इस प्रकार के कृत्यों की पुनरावृत्ति न हो, इस हेतु गृह मंत्रालय भारत सरकार BPRD एवं इलेक्ट्रॉनिकी और सूचना प्रौद्योगिकी मंत्रालय MIEITY की गाइडलाइन उत्तर प्रदेश सरकारी सेवक आचरण नियमावली 1956 यथा संशोधित 1998, 2002, उत्तर प्रदेश पुलिस वर्दी विनियम ऑफीसियल सीक्रेट एक्ट तथा अन्य सुसंगत नियमों के अंतर्गत उ0प्र0 पुलिस के समस्त कार्मिकों हेतु निम्नलिखित निर्देश जारी किये जा रहे हैंः-

सोशल मीडिया के प्लेटफार्म पर प्रतिबंधित गतिविधियाँ

1. कार्य करते समय प्रत्येक पुलिस कार्मिक का यह कर्तव्य है कि वह प्रदत्त कार्यों को पूर्ण लगन एवं मनोयोग से निष्पादित करें। सरकारी कार्य के दौरान सोशल मीडिया का व्यक्तिगत प्रयोग निश्चित रूप से पुलिसकर्मी के बहुमूल्य समय को नष्ट करता है। अतः राजकीय एवं विभागीय हित में इसे प्रतिबंधित किया जाता है।

2. कार्य करते समय अपने कार्यालय एवं कार्यस्थल पर वर्दी में वीडियो / रील्स इत्यादि बनाने अथवा किसी भी कार्मिक द्वारा अपने व्यक्तिगत सोशल मीडिया के प्लेटफार्म पर लाइव टेलीकास्ट को प्रतिबंधित किया जाता है।

3. ड्यूटी के उपरांत बावर्दी किसी भी प्रकार के वीडियो या रील्स इत्यादि, जिससे पुलिस की छवि धूमिल हो, सोशल मीडिया प्लेटफॉर्म पर अपलोड करने को प्रतिबंधित किया जाता है।

4. थाना, कार्यालय इत्यादि के निरीक्षण एवं पुलिस ड्रिल / फायरिंग इत्यादि से संबंधित वीडियो का लाइव टेलीकास्ट एवं सोशल मीडिया पर अपलोड करना गोपनीयता का उल्लंघन है। कार्य सरकार को गोपनीयता बनाये रखने के दृष्टिकोण से इसे प्रतिबंधित किया जाता है।

5. अपने कार्यस्थल से संबंधित किसी वीडियो / रील्स इत्यादि के जरिये शिकायतकर्ता के संवाद का लाइव टेलीकास्ट / वीडियो सोशल मीडिया प्लेटफार्म पर अपलोड करना उसकी निजता का उल्लंघन हो सकता है। अतः सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर इसे प्रतिबंधित किया जाता है।

6. पुलिस कार्मिक द्वारा कार्य करते समय सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर किसी प्रकार की कोचिंग, लेक्चर, लाइव प्रसारण, वेबिनार इत्यादि में आमंत्रित किये जाने पर उसमें भाग लेने से पूर्व अपने वरिष्ठ अधिकारी को सूचित कर अनुमति प्राप्त करना अनिवार्य होगा।

7. सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म से पुलिस कार्मिक किसी भी प्रकार का धनार्जन / आय प्राप्त नहीं करेंगे, जब तक कि इस संबंध में उनके द्वारा सरकार की पूर्व स्वीकृति प्राप्त न कर ली जाये। उत्तर प्रदेश सरकारी सेवक आचरण नियमावली 1956 के नियम 15 में उल्लिखित है कि कोई सरकारी कर्मचारी सिवाय उस दशा में, जब उसने सरकार की पूर्व स्वीकृति प्राप्त कर ली हो, प्रत्यक्षतः या अप्रत्यक्षतः किसी व्यापार या कारोबार में नहीं लगेगा और न ही कोई नौकरी करेगा।

8. सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म से किसी भी व्यक्तिगत व्यवसायिक कम्पनी अथवा उत्पाद सेवा का प्रचार-प्रसार किया जाना प्रतिबंधित किया जाता है।

9. सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर पुलिस कर्मियों द्वारा ऐसी कोई जानकारी साझा नहीं की जाएगी, जो उन्हें अपनी विभागीय नियुक्ति के कारण प्राप्त हुई हो। ऐसी कोई जानकारी तभी साझा की जा सकेगी जब वह कार्मिक इस कार्य के लिए अधिकृत हो।

10. निजता एवं सुरक्षा के कारणों से, सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर पुलिस कार्मिकों द्वारा अपनी अथवा किसी अन्य पुलिस कार्मिक की विशेष नियुक्ति या व्यक्तिगत विवरण का उल्लेख नहीं किया जाएगा। अभिसूचना संकलन किसी गुप्त ऑपरेशन (Under Cover Operation) में संलग्न पुलिस कार्मिकों द्वारा इस प्रावधान का सख्ती से अनुपालन किया जायेगा।

11. अपराध की अन्वेषण, विवेचनाधीन या न्यायालय में लंबित प्रकरणों से संबंधित कोई गोपनीय जानकारी सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर साझा नहीं की जाएगी एवं उन पर कोई टिप्पणी नहीं की जाएगी। उपरोक्त विषय वस्तु पर सक्षम अधिकारी द्वारा अधिकृत या आवश्यक जानकारी सार्वजनिक प्रेस नोट द्वारा साझा की जायेगी।

12. किसी गोपनीय सरकारी दस्तावेज, हस्ताक्षरित रिपोर्ट अथवा दस्तावेज को सरकारी या व्यक्तिगत सोशल मीडिया प्लेटफार्म पर नहीं डाला जायेगा।

13. यौन शोषण पीड़िता या किशोर / किशोरी तथा किशोर आरोपित दोषी (Juvenile Offenders) की पहचान अथवा नाम अन्य संबंधित विवरण सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर उजागर नहीं किया जाएगा।

14. जिन आरोपियों की शिनाख्त परेड बाकी हो, उनका चेहरा सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर सार्वजनिक नहीं किया जाएगा।

15. सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर महिलाओं एवं अनुसूचित जाति / अनुसूचित जनजाति की गरिमा को प्रभावित करने वाले या उनकी गरिमा के विपरीत कोई भी टिप्पणी नहीं की जायेगी।

16. सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर पुलिस कार्मिकों द्वारा पुलिस विभाग, किसी वरिष्ठ अधिकारी या अपने सहकर्मी के विरुद्ध कोई आपत्तिजनक टिप्पणी नहीं की जायेगी, जिससे विभागीय गरिमा प्रभावित हो।

17. पुलिस कार्मिकों द्वारा विभाग में असंतोष की भावना फैलाने वाली पोस्ट अथवा सामग्री सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर साझा नहीं की जाएगी।

18. पुलिस की टैक्सटिक, फील्ड क्राफ्ट, विवेचना या अपराध की अन्वेषण में प्रयुक्त होने वाली तकनीक की जानकारी सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर साझा नहीं की जाएगी।

19. पुलिस कार्मिकों द्वारा सरकार या उसकी नीतियों, कार्यक्रमों, राजनीतिक दल, राजनीतिक व्यक्ति, राजनीतिक विचारधारा एवं राजनेताओं के संबंध में सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर कोई टिप्पणी नहीं की जाएगी।

20. पुलिस कार्मिकों द्वारा अश्लील / हिंसात्मक भाषा का प्रयोग एवं अश्लील फोटो वीडियो सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर पोस्ट अथवा साझा नहीं किया जाएगा।

21. पुलिस कार्मिकों द्वारा सरकारी एवं व्यक्तिगत सोशल मीडिया अकाउंट से की जाने वाली पोस्ट में किसी जाति, धर्म, वर्ग, सम्प्रदाय, व्यवसाय, सेवाएं, संवर्ग, लिंग, क्षेत्र, राज्य इत्यादि के संबंध में भेदभावपूर्ण पूर्वाग्रह या दुराग्रह से ग्रसित कोई टिप्पणी नहीं की जाएगी।

22. राष्ट्रीय सुरक्षा से जुड़े महत्वपूर्ण एवं संवेदनशील प्रकरणों में सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर कोई टिप्पणी नहीं की जाएगी। न्यायालयों द्वारा निर्गत दिशा-निर्देशों का उल्लंघन नहीं किया जाएगा।

23. सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर ऐसी कोई पोस्ट नहीं की जाएगी और न ही ऐसी कोई जानकारी साझा की जाएगी, जिससे माननीय न्यायालयों को अवमानना की स्थिति उत्पन्न हो।

24. पुलिस कार्मिकों द्वारा सरकारी एवं व्यक्तिगत सोशल मीडिया अकाउंट से ऐसे किसी व्यक्ति के साथ फोटो या वीडियो नहीं की जाएगी, जो आपराधिक गतिविधियों में लिप्त हो, या जिसका आपराधिक इतिहास हो।

25. व्यक्तिगत सोशल मीडिया प्लेटफार्म पर पुलिस कार्मिकों द्वारा पृष्ठों का चयन करते समय सतर्कता बरतना अपेक्षित है। पुलिस कार्मिक किसी भी व्यक्ति को फॉलो न करें जो असामाजिक / आपराधिक गतिविधियों में लिप्त हो।

26. पुलिस कार्मिकों द्वारा किसी भी प्रकार के मादक पदार्थों के प्रभाव में तथा मादक पदार्थों के साथ फोटो / वीडियो व्यक्तिगत सोशल मीडिया अकाउंट पर पोस्ट / साझा (Share) नहीं की जाएगी।

27. सराहनीय कार्य से संबंधित पोस्ट में अभियुक्तों की फोटो वीडियो सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर ब्लर करके ही पोस्ट साझा की जाएगी।

28. पुलिस कार्यवाही के दौरान बरामद माल एवं हथियार को बिना सील किए फोटो / वीडियो सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर नहीं डाली जाएगी।

29. पुलिस कार्मिकों द्वारा गश्त / वाहन चेकिंग के दौरान मौके पर मोबाइल से फोटो / वीडियो लेते समय Geo Tagging के विकल्प को बंद रखा जाएगा।

30. गश्त, पेट्रोलिंग या राजकीय कार्यों के निपटान के समय कार्यक्षेत्र में मिलने वाले व्यक्तियों की फोटो / वीडियो आवश्यकता पड़ने पर ब्लर करके ही सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर डाली जाएगी।

31. पुलिस की वर्दी, सरकारी अस्त्र-शस्त्र / वाहन इत्यादि का प्रयोग करते हुए पुलिस कार्मिक के परिजन / मित्रों इत्यादि द्वारा कोई वीडियो या फोटो अपने व्यक्तिगत सोशल मीडिया अकाउंट से अपलोड नहीं किया जाएगा।

32. पुलिस कार्मिकों द्वारा व्यक्तिगत कार्यों, व्यक्तिगत आयोजनों से संबंधित फोटो / वीडियो सरकारी सोशल मीडिया प्लेटफार्म पर पोस्ट / साझा नहीं किया जाएगा।

33. पुलिस कार्मिकों द्वारा भारतीय कॉपीराइट एक्ट, 1957 (The Copyright Act, 1957) का उल्लंघन करने वाली कोई भी पोस्ट, फोटो / वीडियो सरकारी एवं व्यक्तिगत सोशल मीडिया प्लेटफार्म पर अपलोड / साझा नहीं की जाएगी।

34. पुलिस कार्मिकों द्वारा किसी भी प्रकार के सांकेतिक विरोध से संबंधित प्रतीक को सरकारी एवं व्यक्तिगत सोशल मीडिया अकाउंट के डीपी / प्रोफाइल पिक्चर आदि के रूप में नहीं लगाया जाएगा।

35. पुलिस कार्मिक द्वारा सरकारी एवं व्यक्तिगत सोशल मीडिया अकाउंट की डीपी, प्रोफाइल पिक्चर पर किसी भी संगठन या राजनीतिक दल से संबंधित प्रतीक नहीं लगाया जाएगा।

36. पुलिस कार्मिक सरकारी एवं व्यक्तिगत सोशल मीडिया अकाउंट से ऐसे किसी भी वाट्सएप ग्रुप, पेज इत्यादि को ज्वाइन नहीं करेंगे, जो पुलिस विभाग या सरकार के विरोध में हो एवं जाति, सम्प्रदाय, क्षेत्रवाद आदि के नाम पर बनाया गया हो और न ही स्वयं ऐसा कोई ग्रुप बनाएंगे।

37. पुलिस कार्मिक सरकारी सोशल मीडिया अकाउंट को अपने व्यक्तिगत मोबाइल पर लॉगिन नहीं करेंगे। सोशल मीडिया प्लेटफार्म पर व्यक्तिगत अकाउंट बनाते समय ध्यान रखें कि सरकारी मोबाइल नंबर, इंटरनेट, वाईफाई, आईपी एड्रेस, ई-मेल आईडी का प्रयोग नहीं किया जाएगा।

38. सोशल मीडिया प्लेटफार्म पर प्राप्त होने वाली पोस्ट, फोटो / वीडियो को सत्यापन किये बिना अग्रसारित नहीं किया जाएगा एवं किसी भी अफवाह अथवा भ्रामक खबर की पुष्टि हेतु फैक्ट चेक के लिए ट्विटर हैंडल UPPViralCheck, फेसबुक पेज UPPFactCheck एवं इंस्टाग्राम अकाउंट UPPFactCheck पर जानकारी की जा सकती है। इसके साथ ही मुख्यालय के सोशल मीडिया सेंटर से भी इसकी पुष्टि की जा सकती है।

39. कार्मिक द्वारा सोशल मीडिया के माध्यम से सेवा संबंधी प्रकरणों का निस्तारण करने हेतु वीडियो अथवा पोस्ट अपलोड / साझा नहीं की जाएगी। कार्मिकों द्वारा अपने सेवा संबंधी प्रकरणों के निस्तारण हेतु विभागीय प्रक्रिया का पालन किया जाएगा।

40. सोशल मीडिया प्लेटफार्म पर प्रचलित ऑनलाइन पोल / वोटिंग पर किसी भी सरकारी सोशल मीडिया अकाउंट से बिना अनुमति के प्रतिभाग नहीं किया जाएगा और न ही उक्त संबंध में कोई टिप्पणी की जाएगी।

41. पुलिस कार्मिकों द्वारा सोशल मीडिया प्लेटफार्म पर किसी भी व्यक्ति को ट्रोल अथवा बुली (Bullying) नहीं किया जाएगा।

सोशल मीडिया के प्लेटफार्म पर अनुमन्य गतिविधियाँ

प्रत्येक पुलिस कार्मिक एक सामान्य नागरिक के रूप में सोशल मीडिया के प्रयोग एवं उस पर अभिव्यक्ति हेतु उस सीमा तक स्वतंत्र है जहां तक उसकी द्वारा उ0प्र0 सरकारी कर्मचारी आचरण नियमावली 1956 यथा संशोधित 1998, 2002, उत्तर प्रदेश पुलिस वर्दी विनियम, अन्य सुसंगत नियमों तथा सोशल मीडिया पर आचरण एवं नियमावली संबंधी परिपत्रों / निर्देशों का किसी भी प्रकार से उल्लंघन नहीं किया जाता है। पुलिस कार्मिक सोशल मीडिया पर कोई ऐसी पोस्ट न डालें और न ही ऐसा कोई आचरण करें जो मानवीय गरिमा एवं विधिक उपबंधों के प्रतिकूल हो।

पुलिस कार्मिकों द्वारा एक सामान्य नागरिक के रूप में व्यक्तिगत अकाउंट से सोशल मीडिया प्लेटफार्म पर की गई अभिव्यक्ति में यह स्पष्ट किया जाए कि उक्त विचार उनके निजी विचार हैं एवं इससे विभाग का कोई सरोकार नहीं है। सोशल मीडिया प्लेटफार्म पर की जाने वाली टिप्पणी के लिए संबंधित कार्मिक व्यक्तिगत रूप से उत्तरदायी होगा।

पुलिस कार्मिकों द्वारा अपने कार्य सरकार को प्रभावित किए बिना कर्तव्य-निर्वहन जन-सहायता, जनसुरक्षा, जनकल्याणपूर्ण कार्यों एवं व्यक्तिगत उपलब्धियों से संबंधित पोस्ट, फोटो / वीडियो को अपने व्यक्तिगत सोशल मीडिया अकाउंट से साझा किया जा सकता है।

पुलिस कार्मिक राजकीय कार्य से संबंधित ऑनलाइन गतिविधियों जैसे वेबिनार, मीटिंग आदि में वरिष्ठ अधिकारियों को सूचित कर एवं उनसे अनुमति प्राप्त कर सम्मिलित हो सकते हैं। उक्त के अतिरिक्त अन्य ऑनलाइन गतिविधियों के संदर्भ में उ0प्र0 सरकारी कर्मचारी आचरण नियमावली 1956 यथा संशोधित 1998, 2002 के प्रावधान यथावत् लागू रहेंगे, जिसमें उल्लिखित है कि कोई कर्मचारी स्वीकृति प्राप्त किए बिना कोई सामाजिक या धार्मिक प्रकार का अवैतनिक कार्य या कोई साहित्यिक, कलात्मक या वैज्ञानिक प्रकार का आकस्मिक कार्य नहीं कर सकता है, लेकिन शर्त यह है कि इस कार्य द्वारा उसके सरकारी कर्तव्यों में कोई अड़चन नहीं पड़ती है तथा वह उक्त कार्य को हाथ में लेने से एक महीने के भीतर ही अपने विभागाध्यक्ष को और विभागाध्यक्ष के माध्यम से सरकार को सूचना दे देगा। यदि सरकार उसे इस प्रकार का कोई आदेश दे, तो वह ऐसा कार्य हाथ में नहीं लेगा और यदि उसने ऐसा कार्य ले लिया है, तो उसे बंद कर देगा।

पुलिस कार्मिक द्वारा कार्य सरकार को प्रभावित किए बिना अपने व्यक्तिगत सोशल मीडिया अकाउंट से पुलिस के सराहनीय कार्यों से संबंधित पोस्ट को री-ट्वीट / शेयर / लाइक / कमेंट किया जा सकता है।

सोशल मीडिया अकाउंट की सुरक्षा हेतु की जाने वाली गतिविधियाँ

सरकारी एवं व्यक्तिगत सोशल मीडिया अकाउंट की सुरक्षा हेतु स्ट्रांग पासवर्ड बनाएँ और निरंतर उसे बदलते रहें। अकाउंट को सुरक्षित लॉगिन किए जाने हेतु Two-Factor Authentication का प्रयोग किया जाए।

सरकारी एवं व्यक्तिगत सोशल मीडिया अकाउंट के नोटिफिकेशन में किसी भी संदिग्ध लिंक (URL) के प्राप्त होने पर उसे ओपन नहीं करेंगे।

सरकारी सोशल मीडिया अकाउंट में सरकार CUG मोबाइल एवं NIC की सरकारी मेल आईडी का प्रयोग किया जाएगा तथा अकाउंट की सुरक्षा हेतु समय-समय पर मुख्यालय से जारी किए जाने वाले निर्देशों का पालन किया जाएगा।

सरकारी सोशल मीडिया अकाउंट को यथासंभव डेस्कटॉप पर ही चलाया जाए। यदि किन्हीं अपरिहार्य परिस्थितियों में सोशल मीडिया अकाउंट को मोबाइल लॉगिन किया जाता है, तो कार्य समाप्त होने के उपरांत संबंधित सोशल मीडिया अकाउंट को मोबाइल से लॉग आउट कर दिया जाए। सोशल मीडिया अकाउंट ऐसे मोबाइल में ही लॉगिन किया जाए जिसमें थर्ड पार्टी एप्लीकेशन (गेमिंग इत्यादि) न हो।

किसी भी अज्ञात मोबाइल नंबर से किसी अज्ञात व्यक्ति द्वारा फोन कर अपने आपको सोशल मीडिया अकाउंट कंपनी का प्रतिनिधि बताकर संबंधित सोशल मीडिया अकाउंट को वेरीफाई किए जाने का नाम पर यदि लॉगिन किए जाने से संबंधित OTP मांगा जाता है, तो किसी भी दशा में OTP नहीं बताया जाएगा और संबंधित कॉल करने वाले की जानकारी कर उसके विरुद्ध जांचोपरांत गलत पाए जाने पर वैधानिक कार्यवाही की जाएगी।

सरकारी सोशल मीडिया अकाउंट को किसी भी कंप्यूटर अथवा मोबाइल पर लॉगिन करते समय इसकी आई.डी और पासवर्ड को किसी वेब ब्राउज़र पर सेव नहीं किया जाएगा।

सोशल मीडिया पॉलिसी का उल्लंघन करने पर की जाने वाली कार्यवाही

सोशल मीडिया पॉलिसी के उल्लंघन से संबंधित प्रकरण संज्ञान में आने पर सक्षम अधिकारी द्वारा उल्लंघन करने वाले कार्मिक के विरुद्ध आवश्यक कार्यवाही करके, कृत कार्यवाही से अपने पर्यवेक्षण अधिकारियों को अवगत कराया जाएगा। सोशल मीडिया के संदर्भ में भारतीय पुलिस सेवा के अधिकारियों की काडर कंट्रोलिंग अथॉरिटी गृह मंत्रालय भारत सरकार द्वारा समय-समय पर निर्देश निर्गत किए गए हैं। उक्त दिशा-निर्देशों के अतिरिक्त उ0प्र0 पुलिस की सोशल मीडिया पॉलिसी-2023 का पालन करना भी अनिवार्य होगा। निर्देशों का पालन नहीं किए जाने की स्थिति में नियमानुसार आवश्यक कार्यवाही प्रचलित की जाएगी।

(देवेन्द्र सिंह चौहान)
समस्त विभागाध्यक्ष कार्यालयाध्यक्ष
पुलिस विभाग
उत्तर प्रदेश
"""

output5 = """
डीजी परिपत्र संख्या- 06 / 2023

देवेन्द्र सिंह चौहान
आई॰पी॰एस, पुलिस महानिदेशक
उत्तर प्रदेश
पुलिस मुख्यालय, लखनऊ

दिनांकः लखनऊ, फरवरी 14, 2023

विषयः उत्तर प्रदेश गिरोहबन्द तथा समाज विरोधी कियाकलाप निवारण अधिनियम 1986 यथासंशोधित उत्तर प्रदेश गिरोहबन्द तथा समाज विरोधी कियाकलाप (निवारण) संशोधन अधिनियम-2015 तथा उत्तर प्रदेश गिरोहबन्द तथा समाज विरोधी कियाकलाप निवारण नियमावली 2021 के प्रावधानों के अनुपालन के सम्बन्ध में दिशा-निर्देश

प्रिय महोदय / महादया,

उत्तर प्रदेश गिरोहबन्द एवं समाज विरोधी कियाकलाप निवारण नियमावली-2021 उ0प्र0 शासन की अधिसूचना संख्या - 5203 / छः- पु09202131 43-2013 दिनांकित 27.12.2021 द्वारा प्रख्यापित की गयी है, जो तत्काल प्रभाव से लागू है। नियमावली 2021 के प्रावधानों के पूर्ण अनुपालन विधिसम्मत कार्यवाही हेतु अपर पुलिस महानिदेशक, अपराध उत्तर प्रदेश के पत्र संख्या डीजी सात एस 14 09 / 2021 दिनांकित 25.04.2022 तथा अधोहस्ताक्षरी के पत्र संख्या डीजी सात एस-14 09 / 2021 दिनांकित 01.06.2022, डीजी परिपत्र संख्या 40 / 2022 दिनांक 09.12.2022 तथा डीजी परिपत्र संख्या 20 / 2018 दिनांक 06.05.2018 के द्वारा पूर्व में निर्देशित किया जा चुका है।

उत्तर प्रदेश राज्य के कमिश्नरेट जनपदों में गिरोहबन्द अधिनियम 1986 के अन्तर्गत कार्यवाही के क्रम में अभियुक्तों द्वारा अवैध रूप से अर्जित की गयी सम्पत्ति को अधिनियम की धारा 141 के अन्तर्गत कुर्क किया जाता है। अधिनियम की धारा 141 के अन्तर्गत कुर्की आदेश पारित होने के उपरान्त कार्यवाही के सम्बन्ध में गैंगस्टर एक्ट की धारा 161 में निम्नवत प्रावधान है जहाँ धारा की उपधारा 71 में विनिर्दिष्ट अवधि के भीतर कोई अभ्यावेदन न दिया जाये या जिला मजिस्ट्रेट धारा 15 की उपधारा (2) रिपोर्टे अधीन सम्पत्ति को निर्मुक्त नहीं करता है, वहां वह मामले को अपनी कस्टडी के साथ इस अधिनियम के अधीन अपराध के विचारण करने के लिए अधिकृत न्यायालय को निर्दिष्ट करेगा।

अतः कुर्की आदेश के सम्बंध में विचारण न्यायालय से त्वरित आदेश प्राप्त करने हेतु यह आवश्यक है कि सम्बन्धित प्राधिकारियों से समन्वय स्थापित कर यह सुनिश्चित किया जाए कि इस प्रकार की पत्रावलियां समयबद्ध रूप से जिला मजिस्ट्रेट कार्यालय से सम्बन्धित न्यायालय को संदर्भित कर दी जाएँ।

उत्तर प्रदेश गिरोहबन्द तथा समाज विरोधी कियाकलाप (निवारण अधिनियम 1986) की धारा 17 में कुर्क की गयी सम्पत्ति के सम्बन्ध में विशेष न्यायालय द्वारा जांच किये जाने सम्बन्धित निम्नवत प्रावधान है: यदि ऐसी जांच पर न्यायालय इस निष्कर्ष पर पहुंचे कि सम्पत्ति किसी गिरोहबन्द द्वारा इस अधिनियम के अधीन विचारणीय किसी अपराध कार्य के परिणामस्वरूप अर्जित नहीं की गयी थी तो वह उस सम्पत्ति को ऐसे व्यक्ति के पक्ष में निर्मुक्त करने का आदेश देगा जिसकी कब्जे से वह कुर्क की गयी थी। किसी अन्य मामले में न्यायालय सम्पत्ति को कुर्क करके, अधिग्रहण करके या सम्पत्ति पर कब्जा पाने के लिए हकदार व्यक्ति को देकर या अन्य प्रकार से उसका निस्तारण करने का ऐसा आदेश दे सकता है जैसा वह उचित समझे।

अधिनियम की धारा 217 के अन्तर्गत विशेष न्यायालय गिरोहबन्द अधिनियम में होने वाली कार्यवाहियों के सम्बन्ध में उत्तर प्रदेश गिरोहबन्द तथा समाज विरोधी कियाकलाप निवारण नियमावली 2021 के नियम 53 में व्यवस्था दी गयी है, जिसके अनुसार विशेष न्यायालय द्वारा जांच के दौरान सम्बन्धित लोक अभियोजक को प्रकरण से सम्बन्धित अभिलेख न्यायालय के समक्ष प्रस्तुत करने तथा अभियुक्त सम्पत्ति के स्वामी की ओर से प्रस्तुत साक्षियों की प्रतिपरीक्षा का अवसर प्रदान किया जाएगा।

उपरोक्त के दृष्टिगत प्रदेश की कानून-व्यवस्था चुस्त दुरुस्त रखने, अपराध नियंत्रण तथा गिरोह बनाकर आपराधिक कृत्यों के जरिये धनार्जन करने वाले अपराधियों पर नियंत्रण के लिए यह आवश्यक है कि गिरोहबन्द अधिनियम, 1986 के ऐसे प्रकरण जिनमें जिला मजिस्ट्रेट द्वारा अधिनियम की धारा-141 के अन्तर्गत आदेश पारित करने के उपरान्त पत्रावलियाँ सम्बंधित गैंगस्टर कोर्ट को सन्दर्भित की गयी हैं, उनमें सुसंगत आख्या एवं अभिलेख समय पर न्यायालय में कार्य कर रहे लोक अभियोजक को उपलब्ध कराते हुए प्रभावी पैरवी की जाए, जिससे जांच के दौरान अभियुक्त सम्पत्ति स्वामी द्वारा प्रस्तुत किये गये तर्कों का प्रभावी प्रतिवाद किया जा सके।

अतः आप सभी को निर्देशित किया जाता है कि उत्तर प्रदेश गिरोहबन्द तथा समाज विरोधी कियाकलाप निवारण नियमावली 2021 के प्रावधानों विशेष रूप से आपराधिक कृत्यों द्वारा धनार्जन से अर्जित सम्पत्तियों की कुर्की से सम्बन्धित प्रावधानों से अपने अधीनस्थों को अवगत कराएं तथा गिरोहबन्द अधिनियम की धारा 217 के अन्तर्गत विशेष न्यायालय में विचाराधीन सभी प्रकरणों की नियमित समीक्षा करते हुए कुर्क की गयी सम्पत्ति के सम्बन्ध में राज्य के पक्ष में आदेश प्राप्त करने के प्रभावी प्रयास करें, जिससे कुर्क की गयी सम्पत्ति को राज्य के पक्ष में निर्मुक्त करने का आदेश मा0 न्यायालय द्वारा पारित किया जा सके।

उपरोक्त निर्देशों का कड़ाई से अनुपालन सुनिश्चित करायें।

देवेन्द्र सिंह चौहान
समस्त पुलिस आयुक्त उत्तर प्रदेश
पुलिस उपमहानिरीक्षक / वरिष्ठ पुलिस अधीक्षक / पुलिस अधीक्षक,
प्रभारी जनपद उत्तर प्रदेश

प्रतिलिपि निम्नांकित अधिकारियों को सूचनार्थ एवं आवश्यक कार्यवाही हेतु प्रेषित:

1. अपर पुलिस महानिदेशक, कानून व्यवस्था, उ0प्र0 लखनऊ
2. अपर पुलिस महानिदेशक अपराध, उ0प्र0 लखनऊ
3. अपर पुलिस महानिदेशक मानवाधिकार, उ0प्र0 लखनऊ
4. अपर पुलिस महानिदेशक रेलवेज, उ0प्र0 लखनऊ
5. समस्त जोनल अपर पुलिस महानिदेशक उत्तर प्रदेश
6. समस्त क्षेत्रीय पुलिस महानिरीक्षक / पुलिस उपमहानिरीक्षक उत्तर प्रदेश
"""

output6 = """
डीजी परिपत्र संख्या 10/ 2023

पुलिस महानिदेशक,

देवेन्द्र सिंह चौहान

आई॰पी॰एस0 उत्तर प्रदेश

पुलिस मुख्यालय, गामती नगर विस्तार

लखनऊ 226010

दिनांकः मार्च 1 / 2023

प्रिय महोदय,

विषयः रिट (सी) संख्या 1266 / 2023, प्रेमलता मौर्या बनाम उ0प्र॰ राज्य व अन्य मा0 उच्च न्यायालय इलाहाबाद द्वारा पारित आदेश दिनांकित 16.02.2023 क अनूपालन क सम्बन्ध में |

प्रायः यह देखने में आया है कि विभिन्न जनपदों में कतिपय भूमि सम्पत्ति तथा कब्जेदारी सम्बन्धी विवादों में स्थानीय पुलिस द्वारा नियम संगत कार्यवाही न करने क कारण विवादों का स्वरूप और भी विद्रूपित हो जाता है तथा अन्य गंभीर आपराधिक घटनाएँ भी घटित हो जाती हैं। भूमि सम्बन्धी विवादों क निस्तारण का प्रारम्भिक दायित्व राजस्व विभाग का है किन्तु समुचित ज्ञान तथा मार्गदर्शन क अभाव में पुलिस द्वारा नियम विरूद्ध हस्तक्षेप करने से न केवल असहज स्थिति उत्पन्न हो जाती है वरन माननीय न्यायालय द्वारा भी असंतोष प्रकट करते हुये सख्त टिप्पणियाँ की जाती हैं।

उल्लेखनीय है कि इस सम्बन्ध में पूर्व में भी मुख्यालय एवं शासन स्तर पर समय समय पर विस्तृत दिशा-निर्देश निर्गत किये गये हैं। किन्तु ऐसा प्रतीत हो रहा है कि निर्गत स्वतः स्पष्ट निर्देशों का पुलिस कमिश्नरेट तथा जनपदीय पुलिस द्वारा कड़ाई से अनुपालन नहीं किया जा रहा है। जिसके कारण माननीय न्यायालय द्वारा भी इस सम्बन्ध में टिप्पणी की गयी है। इस तरह के प्रकरणों में समुचित पर्यवेक्षण का अभाव भी स्पष्ट परिलक्षित है।

उपर्युक्त विषयक रिट (सी) संख्या 1266 / 2023 प्रेमलता मौर्या बनाम उ0प्र0 राज्य व अन्य मा0 उच्च न्यायालय द्वारा पारित आदेश दिनांकित 16.02.2023 तथा 01.03.2023 में मा॰ न्यायालय द्वारा दिनांक 16.02.2023 का प्रकरण में सुनवाई के दौरान उ0प्र0 पुलिस के कार्मिकों द्वारा निजी व्यक्तियों को अचल सम्पत्ति का कब्जा दिलाने में शामिल होने क अनेक प्रकरण संज्ञान में आने पर असंतोष व्यक्त करते हुये निम्नवत् टिप्पणी की गयी हैः

This Court sitting in Misc. Bench jurisdiction has found that in most of the cases that are filed before this Court, the allegation is against Police Authorities, especially the Station House Officer and the Sub Inspector, In-Charge of the Police Station that they are helping private respondents in taking over forcible possession of the property of the writ petitioners.

The Director General of Police, U.P is directed to issue a fresh Circular in this regard reminding the Police Officials of the earlier Circulars issued by the Police Headquarters, prohibiting the police personnel from interfering in private disputes and to enforce such Circular with all strictness at his command.

मा0 उच्च न्यायालय द्वारा पारित उपरोक्त आदेश के परिप्रेक्ष्य में पूर्व में निर्गत परिपत्रों के क्रियान्वयन की अपेक्षा की जाती है कि भूमि / सम्पत्ति / कब्जेदारी विवादों में नियमों के अनुसार कार्यवाही सुनिश्चित की जायेः

भूमि / सम्पत्ति / कब्जेदारी के विवादों की जानकारी प्राप्त होने पर उसकी गंभीरता का आकलन कर तत्काल वरिष्ठ अधिकारियों को सूचित करते हुए स्थानान्तरण की कार्रवाई राजस्व विभाग / स्थानीय सक्षम मजिस्ट्रेट की उपस्थिति एवं सक्षम आदेश के बाद की जाये। किसी भी दशा में थाना प्रभारी / उप निरीक्षक अथवा किसी पुलिस कर्मी द्वारा कब्जेदारी विवाद का निस्तारण करते हुए कब्जा दिलाने अथवा कब्जा हटवाने की कार्यवाही न की जाये।

स्वयं अनावश्यक हस्तक्षेप न किया जाये: किसी भूमि के विधिक स्वामी के अधिकारों में बिना न्यायालय व मजिस्ट्रेट क आदेश के अनावश्यक हस्तक्षेप न किया जाये। सम्बन्धित पुलिस थाने का दायित्व है कि निरोधात्मक कार्यवाही करते हुए राजस्व विभाग को सूचित कर यथासम्भव विवाद का निराकरण उनके द्वारा करायें। भूमि राजस्व सम्बन्धी विवादों को सावधानी एवं तत्परता पूर्वक चिन्हित कर उनकी आख्या समय से सम्बन्धित उप जिला मजिस्ट्रेट को लिखित रूप में प्रस्तुत करें तथा तहसील दिवस / समाधान दिवस एवं उक्त के अतिरिक्त सम्बन्धित थाने क उप जिला मजिस्ट्रेट से आवश्यक आदेश प्राप्त कर राजस्व एवं पुलिस की संयुक्त टीम गठित कराकर समयबद्ध तरीके से यथासम्भव निस्तारण की कार्यवाही सुनिश्चित करायें।

अन्य विभागों की अधिकारिता से सम्बन्धित भूमि विवादों को भी चिन्हित किया जाये। इसी प्रकार राजस्व विभाग, नगर निगम, स्थानीय विकास प्राधिकरण इत्यादि के माध्यम से भी ऐसे समस्त भूमि सम्बन्धी प्रकरणों जिनमें हिंसा या विवाद की स्थिति हो को भी चिन्हित कराकर भूमि विवाद रजिस्टर में अभिलिखित कराया जाये।

भूमि विवाद रजिस्टर को अद्यावधिक किया जाये: प्रत्येक थाने पर भूमि विवाद के सम्बन्ध में प्रचलित रजिस्टर को अद्यावधिक किया जाये। बीट आरक्षी के माध्यम से समस्त भूमि सम्बन्धी विवादों को चिन्हित कर बीट सूचना अंकित की जाये एवं सीधे प्राप्त प्रार्थना पत्रों व यूपी-112 पर प्राप्त समस्त सूचनाओं को भी बीटवार भूमि विवाद रजिस्टर में अभिलिखित किया जाये।

बीट सूचना / निरोधात्मक कार्यवाहीः प्रत्येक थाने पर सक्रिय बीट व्यवस्था पर भूमि विवाद / रंजिश के सभी मामलों को चिन्हित कर बीट सूचना अंकित करते हुए 107 / 116 द0प्र0सं0 के अन्तर्गत निरोधात्मक कार्यवाही एवं भारी राशि से पाबन्द कराने की कार्यवाही तत्परता से की जाये ताकि किसी प्रकार की गंभीर घटना घटित न हो। पाबन्द व्यक्तियों द्वारा शर्तों का उल्लंघन करने पर सम्बन्धित पक्षकारों के विरूद्ध तत्काल धारा 122 ख द0प्र0सं॰ की आख्या प्रस्तुत कर मुचलका धनराशि जब्त कराई जाये।

तहसील दिवस सम्बन्धित समाधान दिवस आई॰जी0आर0एस0 भूमि विवादों प्रकरणों से विभाग का भूमि विवाद रजिस्टर में अंकन एवं समन्वय त्वरित निस्तारणः आई0जी0आर0एस0 / तहसील राजस्व दिवस एवं समय-समय पर उच्चाधिकारियों को भूमि विवाद से सम्बन्धित जो पत्र प्राप्त होते हैं, उन्हें विवाद रजिस्टर में अंकित किया जाये। इस प्रकार यह प्रक्रिया सतत् चलती रहेगी। उपरोक्त विवाद अधिकारी का समाधान दिवस पर सम्बन्धित पुलिस कर्मचारी को अभिदिष्ट किया जाये एवं प्रत्येक विवाद को तार्किक निष्पत्ति (निस्तारण) तक पहुँचाया जाये। इस हेतु जिलाधिकारी से सामंजस्य स्थापित करते हुए मात्र समाधान दिवस के अतिरिक्त अनुगामी दिवसों हेतु रोस्टर बनाकर समस्या का समाधान कराया जाये।

समाधान दिवस के दौरान थाने निस्तारित प्रकरणों के अतिरिक्त ऐसे भूमि विवाद सम्बन्धी प्रकरणों जिनका थाने पर निस्तारण सम्भव नहीं है, के निस्तारण हेतु जनपद के प्रत्येक थानावार राजस्व एवं पुलिस विभाग की संयुक्त टीमों का गठन किया जाये जिसमें राजस्व एवं पुलिस विभाग के पर्याप्त संख्या में सदस्य सम्मिलित हों। शिकायतों के निस्तारण हेतु जिलाधिकारी एवं वरिष्ठ पुलिस अधीक्षक / पुलिस अधीक्षक क संयुक्त हस्ताक्षर से ग्रामवार / थानावार / तहसीलवार स्थानीय सुविधानुसार मासिक कार्यक्रम निर्धारित किया जाये और उसका अग्रिम रूप से व्यापक प्रचार प्रसार कराया जाये। जनसामान्य को सूचित कर दिया जाये कि सम्बन्धित पक्षकार कार्यक्रमानुसार अपने साक्ष्यों एवं अभिलेखों सहित मौके पर उपस्थित रहें ताकि संयुक्त टीम द्वारा अभिलेखों एवं परीक्षण कर यथासम्भव मौके पर ही न्यायोचित कार्यवाही की जा सके और शिकायत का निस्तारण किया जा सके।

जिलाधिकारी एवं वरिष्ठ पुलिस अधीक्षक / पुलिस अधीक्षक द्वारा संयुक्त टीमों की रवानगी के पूर्व समस्त टीम लीडरों की बैठक आहूत कर मौके पर की जाने वाली कार्यवाही की प्रक्रियात्मक एवं विधिक बिन्दुओं, सावधानियों और की गयी कार्यवाही की रिपोर्टिंग आदि के बारे में ब्रीफिंग की जाये तथा उन्हें निर्देश दिया जाये कि माननीय न्यायालय में विचाराधीन मामलों को छोड़कर प्रकरणों का समाधान ग्राम के सम्भ्रांत व्यक्तियों की उपस्थिति में बातचीत-सुलह-समझौता के माध्यम से विधिक प्रक्रियाओं का पालन करते हुए किया जाये।

संयुक्त टीम समाधान दिवस पर कार्यवाही पूर्ण करने के उपरान्त की गयी कार्यवाही का पूरा विवरण थाने के जी॰डी0 / रोजनामचा में दर्ज किया जाये। अभियान के दौरान मौके पर प्राप्त नई शिकायतों व उन पर की गयी कार्यवाही का विवरण भी रोजनामचा में दर्ज किया जाये।

संयुक्त टीमों द्वारा चिह्नित अति संवेदनशील व अति गंभीर प्रकरणों में जिलाधिकारी / वरिष्ठ पुलिस अधीक्षक / पुलिस अधीक्षक / अपर जिलाधिकारी / पुलिस अधीक्षक एवं उपजिलाधिकारी / क्षेत्राधिकारी द्वारा अपनी उपस्थिति में निराकरण / निस्तारण कराया जाये।

सुस्पष्ट एवं विस्तृत अभिलेखीकरण: जिन-जिन प्रकरणों का समाधान हो जाये अथवा जिनमें समझौता हो जाये, ऐसे प्रकरणों का अभिलेखीकरण इस प्रकार किया जाये कि भविष्य में आवश्यकता पड़ने पर प्रयोग में लाया जा सके। प्रत्येक विवाद के सम्बन्ध में सही एवं गलत पक्ष की स्थिति पूर्णतः अंकित की जाये। यदि उसमें अतिरिक्त सामग्री की आवश्यकता हो तो उसके लिए अतिरिक्त पत्रावलियाँ निर्गत की जाये जिससे उसका भविष्य में प्रयोग किया जा सके।

उपरोक्त न्यायालय इलाहाबाद खण्डपीठ लखनऊ द्वारा 16.02.2023 के अनुक्रम में आपसे अपेक्षा है कि उपरोक्त दिशा-निर्देशों सम्बन्ध में मा0 उच्च न्यायालय द्वारा पारित आदेश दिनांकित 16.02.2023 का कड़ाई से अनुपालन एवं सतत् अनुश्रवण कराना सुनिश्चित करें।

संलग्नकः यथोपरि

देवेन्द्र सिंह चौहान

पुलिस महानिदेशक,

उत्तर प्रदेश|

समस्त जोनल अपर पुलिस महानिदेशक उ0प्र॰ |

समस्त पुलिस आयुक्त उ0प्र0

प्रतिलिपिः समस्त विभागाध्यक्ष उ0प्र॰ पुलिस को संलग्नक सहित सूचनार्थ एवं आवश्यक कार्यवाही हेतु प्रेषित |

प्रतिलिपिः निम्नलिखित को संलग्नक सहित सूचनार्थ एवं आवश्यक कार्यवाही हेतु प्रेषितः-

1. समस्त क्षेत्रीय पुलिस महानिरीक्षक / पुलिस उपमहानिरीक्षक

2. समस्त पुलिस उपमहानिरीक्षक / वरिष्ठ पुलिस अधीक्षक / पुलिस अधीक्षक, जनपद प्रभारी, उ0प्र0

3. पुलिस अधीक्षक विधि प्रकोष्ठ, मुख्यालय पुलिस महानिदेशक उ0प्र॰ लखनऊ |
"""

output7 = """
पुलिस महानिदेशक, उत्तर प्रदेश, लखनऊ

पुलिस भवन, गोमतीनगर विस्तार, लखनऊ 226002

दिनांकः मार्च 20, 2023

डीजी परिपत्र संख्या 11

सेवा में,

पुलिस आयुक्त, गौतमबुद्धनगर / कानपुर / वाराणसी / प्रयागराज / आगरा / गाजियाबाद / लखनऊ

समस्त वरिष्ठ पुलिस अधीक्षक / पुलिस अधीक्षक, उत्तर प्रदेश

समस्त पुलिस अधीक्षक रेलवे, उत्तर प्रदेश

आप अवगत हैं कि भारतीय रेलवे के माध्यम से प्रतिदिन लाखों की संख्या में यात्री उत्तर प्रदेश से होकर आवागमन करते हैं। इन यात्रियों में देश के विभिन्न प्रान्तों के यात्री तथा विदेशी नागरिक भी होते हैं। यदि वर्दीधारी पुलिस कर्मी द्वारा रेल यात्रा के दौरान कोई अनुचित आचरण किया जाता है तो सोशल मीडिया के माध्यम से इस प्रकार के संदेश अल्प समय में लाखों लोगों के संज्ञान में आ जाते हैं।

उत्तर प्रदेश पुलिस अनुशासित बल है तथा इसका एक गौरवशाली इतिहास रहा है।

विगत दिनों ट्रेन में पुलिस कर्मियों एवं टी.टी.ई. के मध्य बिना टिकट ए.सी. कोच में यात्रा को लेकर वाद-विवाद एवं दुर्व्यवहार की घटनाएँ समाचार पत्रों में प्रकाशित हुई हैं तथा सोशल मीडिया पर इस प्रकार के वाद-विवाद और दुर्व्यवहार से संबंधित वीडियो वायरल हुए हैं। ऐसे अनुचित कृत्यों से उत्तर प्रदेश शासन एवं पुलिस विभाग की छवि पर प्रतिकूल प्रभाव पड़ता है।

इस प्रकार की प्रमुख घटनाओं में दिनांकः 10-03-2023 को भागलपुर से जग्गूतवी जाने वाली अमरनाथ एक्सप्रेस ट्रेन में ए.सी. कोच में यात्रा कर रहे पुलिस कर्मियों तथा टी.टी.ई. के मध्य वाद-विवाद एवं दुर्व्यवहार का वीडियो सोशल मीडिया पर वायरल हुआ। इसी प्रकार दिनांकः 14-03-2023 को ट्रेन नम्बर-12355 अर्चना एक्सप्रेस के टी.टी.ई. और पुलिस कर्मियों के मध्य वाद-विवाद तथा मारपीट होने के सम्बन्ध में टी.टी.ई. की सूचना पर थाना जीआरपी प्रतापगढ़ पर मु.अ.सं. 7/2023 धारा 147, 332, 353, 504 भा.द.वि. बनाम अज्ञात पंजीकृत किया गया।

राजकीय कार्यों से यात्रा करने वाले पुलिस कर्मियों को राज्य सरकार द्वारा यात्रा भत्ता प्रदान किया जाता है। अतः प्रत्येक पुलिस कर्मी का यह उत्तरदायित्व है कि वे यात्रा के दौरान ऐसा कोई आचरण न करें जिससे पुलिस विभाग की छवि धूमिल हो।

अतः आपको निर्देशित किया जाता है कि अपने अधीनस्थ नियुक्त समस्त पुलिस कर्मियों को निर्देशित करें कि उनके द्वारा रेल यात्रा नियमानुसार की जाए। किसी पुलिस कर्मी द्वारा रेल यात्रा के नियमों का उल्लंघन किए जाने एवं रेल कर्मियों के साथ दुर्व्यवहार किए जाने पर नियमानुसार विधिक / विभागीय कार्यवाही सुनिश्चित की जाए।

(देवेन्द्र सिंह चौहान)
पुलिस महानिदेशक, उत्तर प्रदेश

प्रतिलिपि: निम्नलिखित को सूचनार्थ एवं आवश्यक कार्यवाही हेतु:

1. अपर पुलिस महानिदेशक, कानून एवं व्यवस्था उत्तर प्रदेश
2. समस्त इकाई विभागाध्यक्ष उत्तर प्रदेश पुलिस
3. अपर पुलिस महानिदेशक रेलवे उत्तर प्रदेश
4. समस्त अपर पुलिस महानिदेशक जोन उत्तर प्रदेश
5. समस्त पुलिस महानिरीक्षक / उप महानिरीक्षक, परिक्षेत्र उत्तर प्रदेश
6. पुलिस महानिरीक्षक रेलवे लखनऊ प्रयागराज
7. पुलिस उपमहानिरीक्षक रेलवे लखनऊ प्रयागराज
"""

output8 = """
देवेन्द्र सिंह चौहान, डीजी परिपत्र सं0 07/2023

पुलिस महानिदेशक

उत्तर प्रदेश

पुलिस मुख्यालय, गोमती नगर विस्तार,

लखनऊ-226010

दिनांकः फरवरी 16, 2023

विषयः- MISCELLANEOUS APPLICATION NO.-1849/2021 IN SPECIAL LEAVE PETITION (CRL.) NO.5191/2021, सत्येंद्र कुमार अंतिल बनाम सीवीआई व अन्य में पारित निर्णय दिनांकित 11.07.2022 के अनुपालन हेतु दिशा-निर्देश

प्रिय महोदय/महोदया,

प्रायः यह देखा गया है कि कुछ विवेचना अधिकारी विवेचना के दौरान गवाहों या संदिग्धों को बुलाने के संबंध में या धारा 174 द.प्र.सं॰ के अंतर्गत की जाने वाली जांच के संबंध में दंड प्रक्रिया संहिता की धारा 41ए, 91, 160 और 175 की विधिक आवश्यकताओं का पालन नहीं करते हैं। दंड प्रक्रिया संहिता की ये धाराएं विवेचना अधिकारियों को किसी भी ऐसे व्यक्ति का मौखिक परीक्षण करने का अधिकार देती हैं, जिसके बारे में माना जाता है कि वह मामले के तथ्यों और परिस्थितियों से परिचित है। धारा-41ए दं॰प्र.सं उन सभी मामलों में जहां व्यक्ति को गिरफ्तारी की आवश्यकता नहीं है, पुलिस अधिकारी के समक्ष पेश होने के लिए नोटिस जारी करने का प्रावधान करती है। धारा-41ए के अनुपालन तथा गवाहों और संदिग्धों को पूछताछ हेतु बुलाने के संबंध में आवश्यक निर्देश पूर्व में शासनादेश तथा परिपत्रों के माध्यम से निर्गत किए गए हैं।

मा0 सर्वोच्च न्यायालय द्वारा मिस. एप्लीकेशन सं0-1849/2021, SLP (Crl.) 5191/2021 सत्येंद्र कुमार अंतिल बनाम सी॰वी॰आई. व अन्य में पारित आदेश दिनांकित 15.07.2022 में दिल्ली पुलिस द्वारा निर्गत Standing Order No. 101/2020 के समान निर्देश निर्गत करने हेतु सभी राज्यों को निम्नवत निर्देशित किया गया है:

" 28. It is also brought to our notice that there are no specific guidelines with respect to the mandatory compliance of Section 41A of the Code. An endeavour was made by the Delhi High Court while deciding Writ Petition (C) No. 7608 of 2017 vide order dated 07.02.2018, followed by order dated 28.10.2021 in Contempt Case (C) No. 430 of 2020, CM Application No. 25054 of 2020, wherein not only the need for guidelines but also the effect of non-compliance towards taking action against the officers concerned was discussed. We also take note of the fact that a standing order has been passed by the Delhi Police viz., Standing Order No. 101 of 2020, which provides for a set of guidelines in the form of procedure for issuance of notices/orders by the police officers. Considering the aforesaid action taken in due compliance with the order passed by the Delhi High Court in Writ Petition (C) No. 7608 of 2017 dated 07.02.2018, this Court has also passed an order in Writ Petition (Crl.) 420 of 2021 dated 10.05.2021 directing the State of Bihar to look into the said aspect of appropriate modification to give effect to the mandate of Section 41A. A recent judgment has also been rendered on the same lines by the High Court of Jharkhand in Cr.M.P. No. 1291 of 2021 dated 16.06.2022.

29. Thus we deem it appropriate to direct all the State Governments and the Union territories to facilitate standing orders while taking note of the standing order issued by the Delhi police i.e., Standing Order No. 109 of 2020, to comply with the mandate of Section 41A. We do feel that this would certainly take care of not only the unwarranted arrests, but also the clogging of bail applications before various Courts as they may not even be required for the offences up to seven years."

मा0 उच्च न्यायालय दिल्ली ने रिट पिटीशन (सी) संख्या-7608/2017 अमनदीप सिंह जौहर बनाम (एनसीटी) दिल्ली राज्य में अपने निर्णय दिनांकित 07.02.2018 द्वारा पुलिस अधिकारियों द्वारा अनिवार्य रूप से पालन की जाने वाली प्रक्रिया की स्पष्ट रूप से व्याख्या की है। मा0 सर्वोच्च न्यायालय द्वारा दिए गए उपरोक्त निर्देशों के अनुपालन में रिट पिटीशन (सी) संख्या-7608/2017 उपरोक्त में मा0 उच्च न्यायालय नई दिल्ली द्वारा पारित आदेश दिनांकित 07.02.2018 में निर्गत निर्देशों का अनुपालन उत्तर प्रदेश पुलिस द्वारा किया जाना अनिवार्य हो गया है, जिसके दृष्टिगत प्रमुख सचिव, गृह, उत्तर प्रदेश शासन द्वारा शासनादेश संख्या: 197पी/छः-पु-3-23-2(291)पी/2022 दिनांकित 23.01.2023 निर्गत किया गया है।

सुसंगत वैधानिक प्रावधान:

रिट पिटीशन (सी) संख्या-7608/2017 अमनदीप सिंह जौहर बनाम (एनसीटी) दिल्ली राज्य के मामले में मा0 उच्च न्यायालय दिल्ली द्वारा पारित निर्णय से सुसंगत विधिक प्रावधान (दं॰प्र॰सं की धारा-41ए, 91, 160 और 175), जो किसी भी व्यक्ति के पुलिस अधिकारी के समक्ष उपस्थिति की आवश्यकता से संबंधित हैं, का उद्धरण सुलभ संदर्भ हेतु निम्नवत अंकित किया जा रहा है:

धारा-41ए दं॰प्र.सं (पुलिस अधिकारी के समक्ष पेश होने की सूचना):

(i) पुलिस अधिकारी, ऐसे सभी मामलों में जिनमें धारा 41 की उपधारा (1) के उपबंधों के अधीन किसी व्यक्ति की गिरफ्तारी अपेक्षित नहीं है, उस व्यक्ति को जिसके विरुद्ध इस बारे में उचित परिवाद किया जा चुका है या विश्वसनीय सूचना प्राप्त हो चुकी है या उचित संदेह विद्यमान है कि उसने संज्ञेय अपराध किया है, उसके समक्ष यह ऐसे अन्य स्थान पर, जो सूचना में विनिर्दिष्ट किया जाए, उपस्थित होने के लिए निर्देश देते हुए सूचना जारी करेगा।

(ii) जहां ऐसी सूचना किसी व्यक्ति को जारी की जाती है, वहां उस व्यक्ति का यह कर्तव्य होगा कि वह सूचना के निर्देशों का अनुपालन करे।

(iii) जहां ऐसा व्यक्ति सूचना का अनुपालन करता है और अनुपालन करता रहता है, वहां उसे सूचना में निर्दिष्ट अपराध के संबंध में तब तक गिरफ्तार नहीं किया जाएगा जब तक लिखित रूप से दर्ज किए जाने वाले कारणों से पुलिस अधिकारी की यह राय न हो कि उसे गिरफ्तार कर लेना चाहिए।

(iv) जहां ऐसा व्यक्ति किसी सूचना के निर्देशों का अनुपालन करने में असफल रहता है, वहां पुलिस अधिकारी के लिए यह विधिपूर्ण होगा कि वह इस निमित्त सक्षम न्यायालय द्वारा पारित किए गए आदेश के अधीन, सूचना में निर्दिष्ट अपराध के लिए गिरफ्तार करे।

धारा 91 दं॰प्र॰सं (दस्तावेज या अन्य चीज पेश करने के लिए समन):

(i) जब कभी कोई न्यायालय या पुलिस थाने का कोई भारसाधक अधिकारी यह समझता है कि किसी विवेचना, जांच, विचारण, या अन्य कार्यवाही के प्रयोजनों के लिए, जो इस संहिता के अधीन ऐसे न्यायालय या अधिकारी के द्वारा या समक्ष हो रही हैं, किसी दस्तावेज या अन्य चीज का पेश किया जाना आवश्यक या वांछनीय है तो जिस व्यक्ति के कब्जे या शक्ति में ऐसे दस्तावेज या चीज के होने का विश्वास है, उसके नाम ऐसा न्यायालय एक समन या ऐसा अधिकारी एक लिखित आदेश उससे यह अपेक्षा करते हुए जारी कर सकता है कि उस समन या आदेश में उल्लिखित समय और स्थान पर उसे पेश करे अथवा हाजिर हो और उसे पेश करे।

(ii) यदि कोई व्यक्ति, जिससे इस धारा के अधीन दस्तावेज या अन्य चीज पेश करने की अपेक्षा की गई है, उसे पेश करने के लिए स्वयं हाजिर होने के बजाय उस दस्तावेज या चीज को पेश करवा दे, तो यह समझा जाएगा कि उसने उस अपेक्षा का अनुपालन कर दिया है।

(iii) इस धारा की कोई बात:

(a) भारतीय साक्ष्य अधिनियम, 1872 (1872 का 2) की धारा 123 और 124 या बैंकर्स बुक्स एविडेंस अधिनियम, 1891 (1891 का 13) पर प्रभाव डालने वाली नहीं समझी जाएगी; अथवा

(b) डाक या तार प्राधिकारी की अभिरक्षा में किसी पत्र, पोस्टकार्ड, तार या अन्य दस्तावेज या किसी पार्सल या चीज को लागू होने वाली नहीं समझी जाएगी।

धारा 160 दं॰प्र॰सं (साक्षियों की हाजिरी की अपेक्षा करने की पुलिस अधिकारी की शक्ति):

(i) कोई पुलिस अधिकारी, जो इस अध्याय के अधीन विवेचना कर रहा है, अपने थाने की या किसी पास के थाने की सीमाओं के अंदर विद्यमान किसी ऐसे व्यक्ति से, जिसकी दी गई सूचना से या अन्यथा उस मामले के तथ्यों और परिस्थितियों से परिचित होना प्रतीत होता है, अपने समक्ष हाजिर होने की अपेक्षा लिखित आदेश द्वारा कर सकता है और वह व्यक्ति अपेक्षानुसार हाजिर होगा।

परंतु किसी पुरुष से [जो पंद्रह वर्ष से कम आयु का या पैंसठ वर्ष से अधिक आयु का है या किसी स्त्री से या किसी मानसिक या शारीरिक रूप से निःशक्त व्यक्ति से] ऐसे स्थान से जिसमें ऐसा पुरुष या स्त्री निवास करती है, भिन्न किसी स्थान पर हाजिर होने की अपेक्षा नहीं की जाएगी।

(ii) अपेक्षानुसार हाजिर होने के लिए कोई भी आदेश इस धारा की उपधारा (1) के अधीन नहीं किया जाएगा जब तक हाजिर होने की अपेक्षा करने वाले पुलिस अधिकारी द्वारा उचित खर्चे का भुगतान नहीं कर दिया गया है, जिसे किसी सक्षम न्यायालय द्वारा निर्धारित किया गया हो।

धारा-175 दं॰प्र॰सं (व्यक्तियों को समन करने की शक्ति):

(i) धारा 174 के अधीन कार्यवाही करने वाला पुलिस अधिकारी उक्त जांच के प्रयोजन से और किसी अन्य ऐसे व्यक्ति को, जो मामले के तथ्यों से परिचित प्रतीत होता है, लिखित आदेश द्वारा समन कर सकता है तथा ऐसा समन किया गया प्रत्येक व्यक्ति हाजिर होने के लिए और उन प्रश्नों के सिवाय, जिनके उत्तरों की प्रवृत्ति उसे आपराधिक आरोप या शास्ति या संपहरण को आशंका में डालने की है, सभी प्रश्नों का सही-सही उत्तर देने के लिए बाध्य होगा।

(ii) यदि तथ्यों से ऐसा कोई संज्ञेय अपराध, जिसे धारा 170 लागू है, प्रकट नहीं होता है तो पुलिस अधिकारी ऐसे व्यक्ति से मजिस्ट्रेट के न्यायालय में हाजिर होने की अपेक्षा नहीं करेगा।

3. नोटिस आदेश जारी करने की प्रक्रिया-

रिट पिटीशन (सी॰ संख्या-7608/2017 अमनदीप सिंह जौहर बनाम (एनसीटी) दिल्ली राज्य के मामले में मा0 उच्च न्यायालय दिल्ली द्वारा पारित निर्णय का परिशीलन करने से यह परिलक्षित होता है कि जहां तक धारा 41ए दं॰प्र.सं के क्रियान्वयन का संबंध है, उ0प्र0 पुलिस के पुलिस अधिकारियों विवेचकों द्वारा निम्न प्रक्रिया का सख्ती से पालन किया जाएगाः

i. पुलिस अधिकारियों द्वारा अनिवार्य रूप से दं॰प्र॰सं॰ के अध्याय-VI में निहित प्रावधानों की शर्तों के अनुसार ही धारा 41ए दं॰प्र.सं के अंतर्गत औपचारिक रूप से नोटिस जारी किया जाना चाहिए। धारा-41ए दं॰प्र.सं॰ के अंतर्गत नोटिस का प्रारूप और इसकी पावती अनुलग्नक A के रूप में संलग्न है।

ii. संबंधित संदिग्ध या आरोपी व्यक्ति के लिए धारा 41ए दं॰प्र.सं के अंतर्गत नोटिस की शर्तों का पालन करना तथा स्वयं को अपेक्षित समय और स्थान पर उपलब्ध कराना अनिवार्य होगा।

iii. यदि आरोपी किसी वैध और न्यायोचित कारण की वजह से दिए गए समय पर स्वयं को प्रस्तुत करने में असमर्थ है, तो आरोपी को तत्काल लिखित रूप में विवेचना अधिकारी को सूचित करना चाहिए और एक उचित अवधि के भीतर वैकल्पिक समय की मांग करनी चाहिए, जो आदर्श रूप से उस तिथि से 04 कार्य दिवसों की अवधि से अधिक नहीं होनी चाहिए, जिस तिथि को उसे उपस्थित होना आवश्यक था जब तक कि वह ऐसी अनुपस्थिति के लिए उचित कारण दिखाने में असमर्थ है।

iv. जब तक यह विवेचना के लिए हानिकारक न हो, पुलिस अधिकारी इस तरह के पुनर्निर्धारण की अनुमति दे सकता है, यद्यपि केसडायरी में उचित कारणों को दर्ज किया जाना चाहिए। यदि विवेचना अधिकारी को यह विश्वास हो जाता है कि विवेचना/जांच में विलंब के लिए इस तरह के अनुरोध की मांग की जा रही है या संदिग्ध या आरोपी व्यक्ति टालमटोल कर रहा है, संबंधित पुलिस स्टेशन के एसएचओ को सूचित करते हुए, इस अनुरोध को अस्वीकार कर उस व्यक्ति को अनिवार्य रूप से उपस्थित होने हेतु निर्देश देना चाहिए।

v. एक संदिग्ध या आरोपी, जो औपचारिक रूप से धारा 41A द.प्र.सं. के अंतर्गत एक नोटिस प्राप्त करता है और पुलिस स्टेशन में जांच या पूछताछ के लिए संबंधित अधिकारी के सामने प्रस्तुत होता है, संबंधित विवेचना अधिकारी से पावती के लिए अनुरोध कर सकता है।

vi. जहां किसी संदिग्ध या आरोपित व्यक्ति को पुलिस थाने से भिन्न किसी स्थान पर उपस्थित होने का निर्देश दिया गया है (जैसा कि धारा 41ए(2) द॰प्र॰सं॰ में व्यवस्था है), उक्त संदिग्ध व्यक्ति उस स्थान पर एक स्वतंत्र साक्षी तथा स्वयं संबंधित विवेचना/जांच अधिकारी द्वारा हस्ताक्षरित पावती (Acknowledgement) प्राप्त करने के लिए स्वतंत्र होगा।

vii. पुलिस थाने के एसएचओ द्वारा जांच अधिकारी को तीन प्रतियों की कार्बन कॉपी प्रारूप में क्रमिक क्रमांकित नोटिस वाली एक विधिवत अनुक्रमित पुस्तिका जारी की जानी चाहिए। नोटिस में अनिवार्य रूप से निम्नलिखित विवरण होने चाहिएः

(a) क्रम संख्या

(b) केस नंबर

(c) उपस्थिति की तिथि और समय

(d) अनुपालन में विफलता की स्थिति में परिणाम

(e) पावती पर्ची

viii. विवेचना जांच अधिकारी निम्नलिखित प्रक्रिया का पालन करेगा:

(a) मूल प्रपत्र आरोपी या संदिग्ध पर तामील किया जाएगा।

(b) एक कार्बन कॉपी श्वेत पत्र पर विवेचना अधिकारी द्वारा अपनी केस डायरी में रखी जाएगी, जिसे आवश्यकता पड़ने पर संबंधित मजिस्ट्रेट को दिखाया जा सकता है।

(c) अन्य पुस्तिकाएं विवेचना अधिकारी द्वारा पुलिस थाने के भारसाधक अधिकारी के पास जमा की जाएंगी, जो विवेचना पूरा होने तक और दं.प्र.सं की धारा 173 (2) के अंतर्गत अंतिम रिपोर्ट प्रस्तुत करने तक इसे अपने पास रखेंगे।

(d) पुलिस विभाग ऐसी पुस्तिकाओं के संरक्षण एवं विनष्ट करने के लिए उपयुक्त नियम बनाएगा।

4. अमनदीप सिंह जौहर बनाम (एनसीटी) दिल्ली राज्य में पारित आदेश दिनांकित 07.02.2018 के माननीय दिल्ली उच्च न्यायालय के अन्य निर्देश:

i. माननीय दिल्ली उच्च न्यायालय ने आगे निर्देश दिया कि उपरोक्त प्रक्रिया दं.प्र.सं की धारा 41ए, 91, 160 और 175 के क्रियान्वयन पर भी लागू होगी। पुलिस द्वारा उपरोक्त सभी धाराओं के क्रियान्वयन की आवश्यकताओं को पूरा करते समय उपरोक्त प्रक्रिया का अनिवार्य रूप से पालन किया जाएगा।

ii. धारा 41ए, 91, 160 और 175 के अंतर्गत जारी किए जाने वाले नोटिस का प्रारूप अनुलग्नक-AB,C और D के रूप में संलग्न किया गया है।

iii. अनुलग्नक E के अनुसार प्रत्येक पुलिस थाने में ड्यूटी अधिकारी द्वारा एक रजिस्टर वर्षवार तैयार किया जाएगा, जिसमें विवेचना जांच अधिकारियों द्वारा जारी नोटिसों का प्रासंगिक विवरण अंकित किया जाएगा।

5. उत्तरदायित्व का क्षेत्र:

i. इस तरह के नोटिस जारी करते समय संबंधित विवेचना/जांच अधिकारी को दिए गए स्थान और समय पर अपनी उपस्थिति सुनिश्चित करनी चाहिए और यदि किसी अप्रत्याशित परिस्थिति या आधिकारिक आवश्यकता के कारण ऐसे अधिकारी का नियत समय व स्थान पर उपस्थित होना संभव न हो तो ऐसी दशा में विवेचना अधिकारी या एसएचओ नोटिस प्राप्तकर्ता के संबंध में आवश्यक कार्यवाही करेंगे। यद्यपि गंभीर प्रकृति की ऐसी किसी दुर्लभ परिस्थिति में जहां उपरोक्त अधिकारियों में से कोई भी उपस्थित न हो, नोटिस प्राप्तकर्ता की उपस्थिति दर्ज करते समय ड्यूटी अधिकारी आवश्यक पावती जारी करेगा और इस आशय का रोजनामचा (जी.डी.) में प्रविष्टि भी करेगा। ड्यूटी अधिकारी नोटिस प्राप्तकर्ता से स्व-प्रमाणित पहचान पत्र (आई.डी.) की एक प्रति भी एकत्र करेगा और तदनुसार विवेचना अधिकारी या एसएचओ को सूचित करेगा। जी.डी. एंट्री और आईडी प्रूफ की कॉपी विवेचना अधिकारी के आने पर उसे सौंप दी जाएगी।

ii. जांच विवेचना अधिकारी अपने द्वारा बुलाए गए व्यक्तियों की व्यक्तिगत सुरक्षा के संबंध में पूरी देखभाल और सावधानी बरतने के लिए जिम्मेदार हैं। उन्हें ऐसे गवाह या संदिग्ध को आत्महत्या करने का प्रयास करने या स्वयं को कोई शारीरिक क्षति पहुंचाने की संभावना से बचाना चाहिए। विवेचना अधिकारी का कार्य जहां तक संभव हो, आगंतुक कक्ष या पुलिस थानों के भूतल में किया जाना चाहिए।

iii. जब किसी महिला से पूछताछ की जानी हो और दं.प्र.सं की धारा 160 के अंतर्गत नोटिस दिया जाना हो, उस स्थिति में विवेचना अधिकारी को यह ध्यान रखना चाहिए कि एक महिला को पुलिस थाने में नहीं बुलाया जा सकता है। हालांकि, नोटिस में विवरण और समय का उल्लेख किया जाएगा जहां महिलाओं से पूछताछ की जाएगी। यह पूछताछ आदर्श रूप से महिला पुलिसकर्मियों की उपस्थिति में की जानी चाहिए।

iv. धारा 160 दं.प्र.सं के अनुसार, पंद्रह वर्ष से कम या पैंसठ वर्ष से अधिक आयु के पुरुष, महिला या मानसिक या शारीरिक रूप से विकलांग व्यक्ति को ऐसे स्थान के अलावा किसी अन्य स्थान पर उपस्थित होने की आवश्यकता नहीं है, जिसमें ऐसा पुरुष निवास करता है। किशोर न्याय अधिनियम के अनुसार, अठारह वर्ष से कम आयु के पुरुष से पूछताछ की जा सकती है जहां वह रहता है और यह पूछताछ आदर्श रूप से उसके परिवार के सदस्यों, संरक्षकों, उपयुक्त व्यक्तियों अथवा किशोर कल्याण अधिकारियों की उपस्थिति में की जानी चाहिए।

v. दं.प्र.सं के प्रावधानों और उपरोक्त प्रक्रिया का पालन करने में विवेचना अधिकारी की ओर से विफलता उसे लागू नियमों के अंतर्गत उचित अनुशासनात्मक कार्यवाही हेतु उत्तरदायी ठहराएगी।

vi. सार्वजनिक समर्थन के विभिन्न तरीकों के माध्यम से जनता को शिक्षित करने के लिए वृहद प्रचार किया जाना चाहिए।

vii. यह स्थायी आदेश उत्तर प्रदेश पुलिस की आधिकारिक वेबसाइट पर भी पोस्ट किया जाएगा ताकि यह सुनिश्चित किया जा सके कि जनता को पालन की जाने वाली प्रक्रिया से अवगत कराया जाए।

viii. उपरोक्त सूचना को पुलिस थानों, अधीनस्थ न्यायालयों और माननीय उच्च न्यायालय में प्रमुख स्थानों पर प्रदर्शित कराया जाना चाहिए तथा राज्य और जिला विधिक सेवा प्राधिकरणों के लिए भी उपलब्ध कराया जाना चाहिए ताकि जनता को उनके अधिकारों और उनके लिए उपलब्ध संसाधनों के विषय में सूचित किया जा सके।

ix. दंड प्रक्रिया संहिता की धारा-41ए, 91, 160 और 175 के प्रभावी अनुपालन के प्रति पुलिस अधिकारियों और न्यायिक अधिकारियों को संवेदनशील बनाने के लिए प्रशिक्षण कार्यक्रम विशेष रूप से तैयार किए जाएं।

7. अभिलेखों को बनाए रखना एवं नष्ट करना

पुलिस थाने के भारसाधक अधिकारी के पास विवेचना अधिकारियों द्वारा जमा की गयी पुरानी पुस्तिकाओं को मामलों के विचारण के दौरान उनकी आवश्यकता के दृष्टिगत विवेचना पूर्ण होने एवं धारा-173(2) व 173(8) दं॰प्र.सं॰ के अन्तर्गत अंतिम रिपोर्ट प्रस्तुत किये जाने के बाद तीन वर्ष तक बनाये रखा जायेगा। यदि अभिलेखों को विनिर्दिष्ट समयावधि के पश्चात भी बनाये रखा जाना हो तो सम्बन्धित सहायक पुलिस आयुक्त या पुलिस उपाधीक्षक की सहमति की आवश्यकता होगी। किसी भी मामले में ऐसे अभिलेखों के अंतिम निस्तारण हेतु सम्बन्धित सहायक पुलिस आयुक्त या पुलिस उपाधीक्षक की सहमति लेनी चाहिए।

8. क्रिमिनल अपील संख्या 838/2021 रिद्धार्थ बनाम (उ0प्र0 राज्य) पारित आदेश 10.08.2021 के मा0 सर्वोच्च न्यायालय ने आरोप पत्र मा0 न्यायालय के समक्ष प्रस्तुत करते समय अभियुक्तों को अनिवार्य रूप से प्रस्तुत किये जाने सम्बन्धी विचारण न्यायालयों द्वारा की जा रही अपेक्षा के बिन्दु पर निर्देश निर्गत किये हैं।

"8. I must say at this stage that the refusal by criminal Courts either through the learned Magistrate or through their office staff to accept the chargesheet without the production of the accused persons is not justified by any provision of law. Therefore, it should be impressed upon all the Courts that they should accept the chargesheet whenever it is produced by the police with any endorsement to be made on the chargesheet by the staff or the Magistrate pertaining to any omission or requirement in the chargesheet. But when the police submit the chargesheet, it is the duty of the Court to accept it especially in view of the provisions of Section 468 of the Code which creates a limitation of taking cognizance of the offence. Likewise, police authorities also should impress on all police officers that if the chargesheet is not accepted for any such reason, then the attention of the Sessions Judge should be drawn to these facts and get suitable orders so that such difficulties would not arise henceforth. We are in agreement with the aforesaid view of the High Courts and would like to give our imprimatur to the said judicial view. It has rightly been observed on consideration of Section 170 of the Cr PC. that it does not impose an obligation on the Officer-in-charge to arrest each and every accused at the time of filing of the chargesheet. We have, in fact, come across cases where the accused has cooperated with the investigation throughout and yet on the chargesheet being filed non-bailable warrants have been issued for his production premised on the requirement that there is an obligation to arrest the accused and produce him before the court. We are of the view that if the Investigating Officer does not believe that the accused will abscond or disobey summons, he/she is not required to be produced in custody. The word 'custody' appearing in Section 170 of the Cr.P.C. does not contemplate either police or judicial custody but it merely connotes the presentation of the accused by the Investigating Officer before the court while filing the chargesheet.

We may note that personal liberty is an important aspect of our constitutional mandate. The occasion to arrest an accused during the investigation arises when custodial investigation becomes necessary or it is a heinous crime or where there is a possibility of influencing the witnesses or accused may abscond. Merely because an arrest can be made because it is lawful does not mandate that arrest must be made. A distinction must be made between the existence of the power to arrest and the justification for exercising it. If arrest is made routine, it can cause incalculable harm to the reputation and self-esteem of a person. If the Investigating Officer has no reason to believe that the accused will abscond or disobey summons and has, in fact, throughout cooperated with the investigation, we fail to appreciate why there should be compulsion on the officer to arrest the accused."

मा0 सर्वोच्च न्यायालय द्वारा पारित उपरोक्त निर्देश के आलोक में आरोप पत्र मा0 विचारण न्यायालयों के समक्ष प्रस्तुत करने हेतु बिना पर्याप्त कारण के मात्र विचारण न्यायालयों द्वारा की जा रही अपेक्षा के क्रम में अभियुक्त को गिरफ्तार कर विचारण न्यायालय के समक्ष प्रस्तुत किये जाने की आवश्यकता नहीं है।

मा0 सर्वोच्च न्यायालय द्वारा मिस. एप्लीकेशन संख्या-1849/2021, SLP Crl. 5191/2021 सतेंद्र कुमार अन्टिल बनाम सी॰बी॰आई॰ व अन्य में पारित आदेश दिनांकित 11.07.2022 में दिये गये निर्देशों के क्रम में सर्वसंबंधित को निर्देशित किया जाता है कि परिपत्र के साथ संलग्न अमनदीप सिंह जोहर बनाम (एनसीटी) दिल्ली राज्य में पारित आदेश दिनांकित 07.02.2018, क्रिमिनल अपील संख्या-1277/2014 अरनेश कुमार बनाम बिहार राज्य व अन्य में पारित आदेश दिनांकित 02.07.2014 तथा क्रिमिनल अपील संख्या 838/2021 सिद्धार्थ बनाम उ0प्र0 राज्य व अन्य में पारित आदेश दिनांकित 16.08.2021 (छायाप्रतियाँ संलग्न) तथा दण्ड प्रक्रिया संहिता के सुसंगत प्रावधानों का अध्ययन करते हुए सुनिश्चित करें कि इन निर्देशों का कड़ाई से अनुपालन कराया जाए। यदि किसी अधिकारी कर्मचारी द्वारा इन निर्देशों के अनुपालन में शिथिलता बरती जाती है तो उनके विरुद्ध नियमानुसार कार्यवाही की जाए।

भवदीय (देवेन्द्र सिंह चौहान)

प्रतिलिपिः निम्नलिखित को अनुपालनार्थ

1. पुलिस आयुक्त, कमिश्ररेट लखनऊ, कानपुर, वाराणसी, गौतमबुद्धनगर, आगरा, गाजियाबाद, प्रयागराज
2. समस्त वरिष्ठ पुलिस अधीक्षक/पुलिस अधीक्षक, प्रभारी जनपद, रेलवेज, उत्तर प्रदेश

प्रतिलिपिः निम्नलिखित को सूचनार्थ एवं आवश्यक कार्यवाही हेतु

1. समस्त पुलिस महानिदेशक, उ0प्र0
2. समस्त जोनल अपर पुलिस महानिदेशक, उ0प्र0
3. समस्त अपर पुलिस महानिदेशक, उ0प्र0
4. समस्त पुलिस महानिरीक्षक/उपमहानिरीक्षक परिक्षेत्र, उ0प्र0

प्रारूप -A

सेवा में, पुलिस स्टेशन

[अभियुक्त/नोटिसी का नाम]

[अंतिम ज्ञात पता]

[फ़ोन नंबर/ ईमेल आईडी (यदि कोई हो)]

धारा 41 (ए) दण्ड प्रक्रिया संहिता के तहत नोटिस

दण्ड प्रक्रिया संहिता की धारा 41ए की उपधारा (1) के तहत प्रदत्त शक्तियों का प्रयोग करते हुए सूचित करता हूं कि मु॰अ॰सं. हए, में आपको अन्तर्गत धारा थाना विवेचना के दौरान यह पता चला है कि वर्तमान जांच के संबंध में तथ्यों और परिस्थितियों का पता लगाने के लिए आपसे पूछताछ करने के लिए उचित आधार हैं।

अतः आपको निर्देशित किया जाता है कि आप मेरे समक्ष दिनांक का पूर्वाह्न/अपराह्न बजे थाना पर उपस्थित होना सुनिश्चित करें। आपको निम्नलिखित निर्देशों का पालन करने का निर्देश दिया जाता हैः

1. आप भविष्य में कोई अपराध नहीं करेंगे।
2. आप मामले में सबूतों के साथ किसी भी तरह से छेड़छाड़ नहीं करेंगे।
3. आप मामले के तथ्य से परिचित किसी भी व्यक्ति को अदालत या पुलिस अधिकारी को ऐसे तथ्यों का खुलासा करने से रोकने के लिए कोई धमकी, प्रलोभन या वादा नहीं करेंगे।
4. जब भी आवश्यक/निर्देशित होगा, आप न्यायालय के समक्ष उपस्थित होंगे।
5. जब भी आवश्यकता होगी आप मामले की जांच में शामिल होंगे और जांच में सहयोग करेंगे।
6. आप मामले के सही निष्कर्ष तक पहुंचने के लिए जांच के उद्देश्य से प्रासंगिक किसी भी हिस्से को छुपाए बिना सभी तथ्यों का सच्चाई से खुलासा करेंगे।
7. आप जांच के प्रयोजन के लिए आवश्यक सभी प्रासंगिक दस्तावेज/सामग्रियां प्रस्तुत करेंगे।
8. आप साथी को पकड़ने में अपना पूरा सहयोग/सहायता प्रदान करेंगे।
9. आप किसी भी तरह से मामले की जांच/परीक्षण के उद्देश्य से प्रासंगिक किसी भी सबूत को नष्ट करने की अनुमति नहीं देंगे।
10. कोई अन्य शर्तें जो मामले के तथ्यों के अनुसार जांच अधिकारी/एसएचओ द्वारा लगाई जा सकती हैं।

इस नोटिस की शर्तों के अनुसार उपस्थित होने/अनुपालन की विफलता की स्थिति में आपको दण्ड प्रक्रिया संहिता धारा 41ए(3) और 4 के तहत गिरफ्तारी के लिए उत्तरदायी बना सकती है।

[नाम और पदनाम]

[चिह्न/मुहर]

दण्ड प्रक्रिया संहिता धारा 41ए के तहत जारी उपर्युक्त नोटिस दिनांक के अनुपालन में नोटिसी/प्राप्तकर्ता दिनांक को समय से तक उपस्थित हुआ है। पुलिस थाने द्वारा रखे गए रजिस्टर में नोटिसी की उपस्थिति दर्ज की गई है।

यह पावती दण्ड प्रक्रिया संहिता की धारा 41ए के अनुपालन में जारी की जा रही है। नोटिसी द्वारा प्रस्तुत दस्तावेजों को जब्ती ज्ञापन/प्रस्तुति ज्ञापन (प्रतिलिपि संलग्न) के माध्यम से विधिवत जब्त कर लिया गया है। नोटिस प्राप्त करने वाला किसी भी अन्य नोटिस का पालन करना जारी रखने का वचन देता है जो उसे वर्तमान जांच के दौरान प्राप्त हो सकता है।

[अभियुक्त के हस्ताक्षर] नोटिसी [विवेचक के हस्ताक्षर]

प्रारूप -B

सेवा में, पुलिस स्टेशन

[अभियुक्त/नोटिसी का नाम]

[अंतिम ज्ञात पता]

[फ़ोन नंबर/ईमेल आईडी (यदि कोई हो)]

धारा 91 दण्ड प्रक्रिया संहिता के तहत नोटिस

जबकि मुझे यह प्रतीत होता है कि दिनांक को थाना में पंजीकृत मु.अ.सं. अन्तर्गत धारा के मामले में जांच के उद्देश्य के लिए नीचे उल्लिखित दस्तावेजों/वस्तुओं की प्रस्तुति आवश्यक या वांछनीय है।

अतः आपको निर्देश दिया जाता है कि आवश्यक सामग्री/दस्तावेज अधोहस्ताक्षरी के समक्ष दिनांक समय तक स्थान पर प्रस्तुत करें।

दस्तावेजों का विवरण:

1.
2.
3.
4.

इस नोटिस की शर्तों के अनुसार उपस्थिति होने/अनुपालन करने में विफल रहने पर आप भारतीय दण्ड संहिता की धारा 174 के तहत कानूनी कार्यवाही के लिए उत्तरदायी हो सकते हैं।

[नाम और पदनाम]

[चिह्न/मुहर]

दण्ड प्रक्रिया संहिता की धारा 91 के तहत जारी उपर्युक्त नोटिस दिनांक के अनुपालन में नोटिसकर्ता दिनांक को समय से तक उपस्थित हुआ है तथा इस हेतु पुलिस थाने पर रखे गए रजिस्टर में नोटिसी की उपस्थिति दर्ज की गई है।

यह पावती दण्ड प्रक्रिया संहिता की धारा 91 के अनुपालन में जारी की जा रही है। इस नोटिस के जारी होने पर प्रस्तुत दस्तावेजों को जब्ती ज्ञापन/प्रस्तुति ज्ञापन (प्रतिलिपि संलग्न) के माध्यम से विधिवत जब्त कर लिया गया है।

नोटिस प्राप्त करने वाला यह वचन देता है कि वह वर्तमान जांच के दौरान प्राप्त होने वाले किसी भी अन्य नोटिस का पालन करना जारी रखेगा।

[अभियुक्त के हस्ताक्षर] नोटिसी [विवेचक के हस्ताक्षर]

प्रारूप -C

सेवा में, पुलिस स्टेशन

[अभियुक्त/नोटिसी का नाम]

[अंतिम ज्ञात पता]

[फ़ोन नंबर/ईमेल आईडी (यदि कोई हो)]

धारा 160 दण्ड प्रक्रिया संहिता के तहत नोटिस

दण्ड प्रक्रिया संहिता की धारा 160 की उप-धारा (1) के तहत प्रदत्त शक्तियों का प्रयोग करते हुए, मैं आपको सूचित करता हूं कि थाना में पंजीकृत एफआईआर मामला संख्या अन्तर्गत धारा दिनांकित की जांच/विवेचना के दौरान यह पता चला है कि वर्तमान जांच के संबंध में तथ्यों एवं परिस्थितियों का पता लगाने के लिए आपसे सवाल करने के लिए उचित आधार हैं।

अतः आपको निर्देशित किया जाता है कि आप मेरे समक्ष दिनांक का पूर्वाह्न/अपराह्न थाना पर उपस्थित होना सुनिश्चित करें।

आपको निम्नलिखित निर्देशों का पालन करने का निर्देश दिया जाता हैः

1. आप आवश्यकता पड़ने पर न्यायालय के समक्ष उपस्थित होंगे।
2. जब भी आवश्यकता होगी आप मामले की जांच में शामिल होंगे और जांच में सहयोग करेंगे।
3. आप मामले के सही निष्कर्ष तक पहुंचने के लिए जांच के उद्देश्य से प्रासंगिक किसी भी हिस्से को छुपाए बिना सभी तथ्यों का सच्चाई से खुलासा करेंगे।
4. आप जांच के उद्देश्य के लिए आवश्यक सभी प्रासंगिक दस्तावेज/सामग्री प्रस्तुत करेंगे।
5. आप साथी को पकड़ने में अपना पूरा सहयोग/सहायता प्रदान करेंगे।
6. आप किसी भी तरह से मामले की जांच/परीक्षण के उद्देश्य से प्रासंगिक किसी भी सबूत को नष्ट करने की अनुमति नहीं देंगे।
7. कोई अन्य शर्तें, जो मामले के तथ्यों के अनुसार जांच अधिकारी/एसएचओ द्वारा लगाई जा सकती हैं।

इस नोटिस की शर्तों के अनुसार उपस्थिति होने/अनुपालन करने में विफल रहने पर आप भारतीय दण्ड संहिता की धारा 174 के तहत कानूनी कार्यवाही के लिए उत्तरदायी हो सकते हैं।

[हस्ताक्षर]

[नाम और पदनाम]

[चिह्न/मुहर]

उपर्युक्त द.प्र॰सं॰ की धारा 160 के तहत जारी नोटिस दिनांक के अनुपालन में नोटिसकर्ता दिनांक को समय तक उपस्थित हुआ तथा इस हेतु पुलिस थाने पर रखे गए रजिस्टर में नोटिसी की उपस्थिति दर्ज की गई है।

यह पावती दण्ड प्रक्रिया संहिता की धारा 160 के अनुपालन में जारी की जा रही है। नोटिसी द्वारा प्रस्तुत दस्तावेजों को जब्ती ज्ञापन/प्रस्तुति ज्ञापन (प्रतिलिपि संलग्न) के माध्यम से विधिवत जब्त कर लिया गया है।

नोटिस प्राप्त करने वाला यह वचन देता है कि वह वर्तमान जांच के दौरान प्राप्त होने वाले किसी भी अन्य नोटिस का पालन करना जारी रखेगा।

[अभियुक्त के हस्ताक्षर] नोटिसी [विवेचक के हस्ताक्षर]

प्रारूप -D

सेवा में, पुलिस स्टेशन

[अभियुक्त/नोटिसी का नाम]

[अंतिम ज्ञात पता]

[फ़ोन नंबर/ईमेल आईडी (यदि कोई हो)]

धारा 175 दण्ड प्रक्रिया संहिता के तहत नोटिस

जबकि दिनांक को थाना पंजीकृत प्रथम सूचना रिपोर्ट संख्या अन्तर्गत धारा के मामले में किए गए अपराध की जांच के प्रयोजन के लिए आपकी उपस्थिति की आवश्यकता है।

अतः आपको एतद्वारा निर्देशित किया जाता है कि दिनांक को पूर्वाह्न/अपराह्न थाना पर अधोहस्ताक्षरी के समक्ष उपस्थित होकर कारित अपराध के सम्बन्ध में जो भी सूचना आपके पास हो, उपलब्ध कराना सुनिश्चित करें।

इस नोटिस की शर्तों के अनुसार उपस्थित होने/अनुपालन करने में विफल रहने पर आप भारतीय दण्ड संहिता की धारा 174 के तहत कानूनी कार्यवाही के लिए उत्तरदायी हो सकते हैं।

[नाम और पदनाम]

[चिह्न/मुहर]

क्रमांक

दण्ड प्रक्रिया संहिता की धारा 175 के तहत जारी उपर्युक्त नोटिस दिनांक के अनुपालन में नोटिसकर्ता दिनांक को समय से तक उपस्थित हुआ है। इस हेतु पुलिस थाने पर रखे गए रजिस्टर में नोटिसी की उपस्थिति दर्ज की गई है।

यह पावती दण्ड प्रक्रिया संहिता की धारा 175 के अनुपालन में जारी की जा रही है। नोटिस प्राप्त करने वाला यह वचन देता है कि वह वर्तमान जांच के दौरान प्राप्त होने वाले किसी भी अन्य नोटिस का पालन करना जारी रखेगा।

[अभियुक्त के हस्ताक्षर] नोटिसी [विवेचक के हस्ताक्षर]

प्रारूप -E

नोटिस निर्गत रजिस्टर

क्रम संख्या

प्रथम सूचना रिपोर्ट सं॰ जी.डी. नं॰

अन्तर्गत धारा

नोटिस अन्तर्गत धारा 41-ए, 91, 160, 175

बुकलेट क्रम संख्या, विवेचक का नाम व मो. नं.

व्यक्ति का नाम, पता, मो. नं. जिसे विवेचक द्वारा बुलाया गया है।

उपस्थिति एवं अनुपालन हेतु दिया गया दिनांक व समय

उपस्थिति एवं अनुपालन हेतु दिया गया वास्तविक दिनांक व समय

नोटिस के निर्देशों के अनुपालन में असफल होने पर की गई कार्यवाही

नोटिसी के उपस्थिति एवं विवेचक को सूचित करने का जी.डी. नं. (यदि विवेचक उपलब्ध न हो)
"""

output9 = """
देवेन्द्र सिंह चौहान

डीजी परिपत्र संख्या 08/2023

आई.पी.एस.

पुलिस महानिदेशक,

उत्तर प्रदेश

पुलिस मुख्यालय, लखनऊ

उत्तर प्रदेश

दिनांक: लखनऊ: मार्च 01, 2023

विषय: रिट याचिका संख्या 156/2016 महेन्द्र चावला बनाम यूनियन ऑफ इण्डिया व अन्य में मा. सर्वोच्च न्यायालय द्वारा पारित निर्णय दिनांक 05.12.2018 के अनुपालन में विटनेस प्रोटेक्शन स्कीम 2018 तथा क्रिमिनल मिस जमानत प्रार्थना पत्र संख्या: 20591/2021 रविन्द्र प्रताप शाही उर्फ पप्पू शाही बनाम उ.प्र. राज्य में मा. उच्च न्यायालय इलाहाबाद द्वारा पारित आदेश दिनांक 07.06.2021 का अनुपालन कराय जाने तथा व्यापक प्रचार-प्रसार करते हुए जनपद/थाना स्तर पर पुलिस कर्मियों को जागरूक किये जाने एवं प्रभावी क्रियान्वयन हेतु दिशा निर्देश।

प्रिय महोदय / महोदया,

आप अवगत हैं कि विवेचनाधीन एवं मा. न्यायालयों में विचाराधीन प्रकरणों से सम्बन्धित किसी भी साक्षी के साथ कोई घटना घटित हो जाने के पश्चात न केवल अभियोजन प्रक्रिया प्रभावित होती है वरन पुलिस की सुरक्षा प्रणाली पर भी प्रश्न चिन्ह लगता है।

शासनादेश सं. 307/छः-पु0-919/31/16/2019, दि. 22.02.19

शासनादेश सं. 3032/2/छः-पु0-931/122/2019, दि. 11.10.19

अर्द्ध-शा. पत्र सं. 2490/छः-पु0-919/31(16)/2019, दि. 27.08.19

कार्यालय ज्ञाप सं. 2359/छः-पु0-919/31(16)/2019, दि. 28.07.19

पत्र सं. डीजी परिपत्र सं. 15/2019, दिनांक 04.04.2019

सं. 3415/छः-पु09-20-31(16)/2019, दि. 14.12.20

पत्र सं. डीजी सात-एस-14/2019, दि. 19.12.2020

पत्र सं. डीजी परिपत्र संख्या 21/2021, दि. 24.06.2021

रिट याचिका संख्या 156/2016 महेन्द्र चावला बनाम यूनियन ऑफ इण्डिया व अन्य में मा. सर्वोच्च न्यायालय द्वारा पारित निर्णय दिनांक 05.12.2018 के अनुपालन में साक्षी सुरक्षा योजना 2018 का क्रियान्वयन हेतु शासन स्तर से समय-समय पर विस्तृत शासनादेश तथा इस मुख्यालय स्तर से परिपत्र निर्गत किये गये हैं जिनका उल्लेख पार्श्वाकिंत बॉक्स में किया गया है।

उपर्युक्त क्रम में क्रिमिनल मिस बेल प्रार्थना पत्र संख्या-20591/2021 रविन्द्र प्रताप शाही उर्फ पप्पू शाही बनाम उ.प्र. राज्य में पारित आदेश दिनांकित 06.07.2021 में साक्षी सुरक्षा योजना 2018 का व्यापक प्रचार-प्रसार करने और योजना की जानकारी प्रत्येक थाना स्तर तक पहुंचाने के लिये योजनाबद्ध प्रयास करने के निर्देश दिये गये हैं। इसके सम्बन्ध में इस मुख्यालय द्वारा निर्गत डीजी परिपत्र 21/2021 में यथोचित निर्देश प्रेषित किये जा चुके हैं।

मा. उच्च न्यायालय द्वारा दिये गये निर्देशों के अनुपालन में अपर मुख्य सचिव उ.प्र. शासन न पत्र संख्या:2309/छः-पु09-21-31(16)/2019, दिनांकित 18.06.2021 द्वारा साक्षी सुरक्षा योजना 2018 का प्रचार-प्रसार एवं प्रभावी क्रियान्वयन हेतु विस्तृत दिशा निर्देश निर्गत किये गये हैं। उ.प्र. शासन द्वारा निदेशक सूचना विभाग उ.प्र. को भी साक्षी सुरक्षा योजना-2018 का वृहद प्रचार-प्रसार सुनिश्चित करने का निर्देश पत्र संख्या:2308/छः-पु0-9-21-31(16)/2019, दिनांकित 18.06.2021 द्वारा निर्गत किये हैं।

अतः मा. उच्चतम न्यायालय द्वारा पारित निर्णय दिनांक 05.12.2018 एवं मा. उच्च न्यायालय इलाहाबाद द्वारा निर्गत आदेश तथा उ.प्र. शासन द्वारा निर्गत निर्देशों के अनुपालन में साक्षी सुरक्षा योजना 2018 का पूर्ण मनोयोग से अनुपालन कराने, इसके प्रति जागरूकता बढ़ाने, योजना की जानकारी प्रत्येक थाने तक पहुंचाने तथा पुलिस कर्मियों को साक्षी सुरक्षा के प्रति संवेदनशील बनाने हेतु निम्नवत् निर्देश निर्गत किये जा रहे हैं:

साक्षी सुरक्षा योजना 2018 के प्रस्तर 2सी का अनुपालन सुनिश्चित किये जाने हेतु गृह (पुलिस) अनुभाग 9 उ.प्र. शासन लखनऊ के आदेश दिनांक 28.07.2019 के क्रम में समस्त जनपदों में जिला एवं सत्र न्यायाधीश की अध्यक्षता में जिलाधिकारी/अभियोजन प्रमुख सम्बन्धित जनपद सदस्य सचिव व जनपद पुलिस प्रमुख सदस्य को सम्मिलित करते हुए स्टैण्डिंग कमेटी का गठन किया गया है। उक्त कमेटी द्वारा साक्षी सुरक्षा स्कीम के अन्तर्गत विटनेस प्रोटेक्शन से सम्बन्धित आवेदन प्राप्त होने के उपरान्त साक्षियों की सुरक्षा हेतु समुचित आदेश पारित किया जायेगा। इस साक्षी सुरक्षा सम्बन्धी आदेश साक्षी सुरक्षा सेल के माध्यम से लागू कराया जायेगा। समस्त जनपदों में गठित उक्त स्टैण्डिंग कमेटी के द्वारा पारित साक्षी सुरक्षा सम्बन्धी आदेशों का पूर्ण अनुपालन सुनिश्चित कराया जाये। कमेटी के समक्ष प्राप्त समस्त आवेदनों की समयबद्ध रूप से समीक्षा करते हुए साक्षियों की समुचित सुरक्षा सुनिश्चित करायी जाये।

साक्षी सुरक्षा योजना 2018 को पूर्ण रूप से लागू किये जाने के क्रम में शासन एवं इस मुख्यालय द्वारा निर्गत शासनादेशों एवं परिपत्रों के परिप्रेक्ष्य में विटनेस प्रोटेक्शन हेतु प्राप्त समस्त आवेदन पत्रों का जनपद के जिला एवं सत्र न्यायाधीश की अध्यक्षता में उपरोक्त वर्णित गठित कमेटी की बैठक में प्रस्तुत किये जाने हेतु जिला एवं सत्र न्यायाधीश से बैठक के लिए समय का अनुरोध करते हुए उन आवेदन पत्रों पर विटनेस प्रोटेक्शन आर्डर पारित कराकर लागू कराय जाने की कार्यवाही सुनिश्चित की जाये। यह भी सुनिश्चित किया जाये कि उक्त बैठक नियमित रूप से हो सके।

साक्षी सुरक्षा योजना 2018 के अन्तर्गत जनपद/कमिश्नरेट स्तर पर गठित विटनेस प्रोटेक्शन सेल के कर्मियों तथा कार्यों की समीक्षा जनपद पुलिस प्रभारी द्वारा नियमित रूप से की जाये। विटनेस प्रोटेक्शन सेल का प्रभारी जनपद में किसी वरिष्ठ अधिकारी को बनाया जायेगा, जो विटनेस प्रोटेक्शन आर्डर के अनुरूप साक्षी की सुरक्षा व्यवस्था सुनिश्चित किये जाने हेतु उत्तरदायी होगा।

एमपी/एमएलए न्यायालयों में विचाराधीन गम्भीर अपराधों से सम्बन्धित वादों के साक्षियों/गवाहों की सुरक्षा के सम्बन्ध में मूल्यांकन एवं विश्लेषण करते हुए साक्षी सुरक्षा योजना 2018 के प्रावधानों के अनुसार यथोचित सुरक्षा उपलब्ध करायी जाये।

साक्षी सुरक्षा योजना-2018 के अनुच्छेद-12 के अनुसार इसका व्यापक प्रचार-प्रसार किया जाये। योजना की मुख्य विशेषताओं से साक्षियों को अवगत कराया जाये। अन्वेषण अधिकारियों/विवेचकों का इस सम्बन्ध में दायित्व निर्धारित किया जाये कि वे इसकी मुख्य विशेषतायें समस्त साक्षियों को बतायें। किसी भी साक्षी द्वारा यदि खतरे की आंशका व्यक्त की जाती है तो इससे तत्काल वरिष्ठ अधिकारियों को संज्ञानित कराते हुए उसे उचित सुरक्षा प्रदान की जाये। साक्षियों की सुरक्षा में किसी प्रकार की शिथिलता या लापरवाही परिलक्षित होने पर सम्बन्धित पुलिस अधिकारियों का उत्तरदायित्व निर्धारित किया जाये। समस्त जोनल अपर पुलिस महानिदेशक एवं पुलिस आयुक्त द्वारा प्रति माह इसकी गहन समीक्षा व कार्यवाही सुनिश्चित करते हुए इस मुख्यालय को भी अवगत कराया जाये।

किसी भी साक्षी का न्यायालय में साक्ष्य/जिरह की कार्यवाही पूर्ण हो जाने के पश्चात भी उसके सुरक्षा के सम्बन्ध में साक्षी सुरक्षा योजना 2018 के अन्तर्गत गठित कमेटी द्वारा आसन्न संकट आंकलन के अनुसार सुरक्षा उपलब्ध कराये जाने हेतु विचार करते हुए सुरक्षा उपलब्ध करायी जा सकती है।

साक्षी सुरक्षा योजना के प्रावधानों के प्रति जागरुकता बढ़ाने तथा फील्ड स्तर पर पुलिस कर्मियों को साक्षी सुरक्षा योजना के प्रति संवेदनशील बनाने हेतु समस्त जोनल अपर पुलिस महानिदेशकों को नोडल अधिकारी के रूप में दायित्व दिया जाता है। अपर पुलिस महानिदेशक अपने जोन के प्रत्येक जनपद में साक्षी सुरक्षा योजना 2018 के प्रावधान एवं अपनायी जाने वाली प्रक्रिया से परिचित कराने हेतु एक कार्यशाला का आयोजन करेंगे। इस कार्यशाला का एक सत्र जोनल अपर पुलिस महानिदेशक द्वारा सम्बोधित किया जायेगा। समस्त पुलिस कमिश्नरेट उक्त कार्यवाही पुलिस आयुक्त के नेतृत्व में सम्पादित की जाये।

साक्षी सुरक्षा योजना 2018 में अंकित निर्देशों तथा साक्षियों के अधिकार के सम्बन्ध में समस्त जनपदों के प्रत्येक थाने में दृष्टव्य स्थान पर एक नोटिस बोर्ड लगाया जायेगा, जिसमें साक्षी सुरक्षा योजना 2018 के अन्तर्गत सुरक्षा हेतु आवेदन करने की प्रक्रिया तथा सम्बन्धित अधिकारियों का दूरभाष नम्बर बड़े अक्षरों में अंकित किया जायेगा जिससे थाने में आने वाले पीड़ित एवं जनसाधारण व्यक्ति योजना के प्रावधानों से परिचित हो सकें और योजना का लाभ प्राप्त कर सकें।

साक्षी सुरक्षा योजना 2018 का विधिवत प्रचार-प्रसार कराये जाने व समस्त कर्मियों की इस सम्बन्ध में ब्रीफिंग किये जाने हेतु जोन/परिक्षेत्र/जनपद/कमिश्नरेट स्तर पर राजपत्रित अधिकारियों को नोडल अधिकारी नियुक्त किया जाये। यह सुनिश्चित किया जाये कि प्रत्येक थाने में ड्यूटी ब्रीफिंग के समय थाना प्रभारी द्वारा साक्षी सुरक्षा योजना 2018 के मुख्य बिन्दुओं को थाने के समस्त कार्मिकों को पढ़कर सुनाया जायेगा तथा इसका उल्लेख जनरल डायरों में किया जायेगा।

साक्षी सुरक्षा योजना-2018 से सम्बन्धित कार्यवाही की समीक्षा जोन स्तर पर प्रतिमाह, परिक्षेत्र स्तर पर पाक्षिक तथा जनपद स्तर पर प्रति सप्ताह की जाये।

साक्षी सुरक्षा योजना 2018 को प्रभावी रूप से लागू करने हेतु उ.प्र. शासन तथा पुलिस महानिदेशक उ.प्र. की ओर से जो शासनादेश एवं परिपत्र निर्गत किये गये हैं, उनका कड़ाई से अनुपालन किया जा रहा है अथवा नहीं, इसकी नियमित समीक्षा परिक्षेत्रीय पुलिस महानिरीक्षक/पुलिस उप महानिरीक्षक, जोनल अपर पुलिस महानिदेशक तथा पुलिस आयुक्त द्वारा की जायेगी।

अतः साक्षी सुरक्षा योजना-2018 एवं इससे सम्बन्धित मा. उच्चतम न्यायालय/उच्च न्यायालय द्वारा पारित समस्त आदेशों का गहन परिशीलन करते हुए प्रभावी कार्यवाही सुनिश्चित करायी जाये। साक्षियों की सुरक्षा हेतु निर्गत समस्त शासनादेशों व परिपत्रों का अध्ययन कर साक्षी सुरक्षा को उच्च प्राथमिकता प्रदान करते हुए निष्पक्ष व न्यायपूर्ण अभियोजन की कार्यवाही सुनिश्चित की जाये जिससे किसी भी साक्षी के साथ कोई भी अप्रिय घटना घटित न होने पाय तथा इस योजना का उचित लाभ साक्षियों को प्राप्त हो सके। इसमें किसी भी प्रकार की शिथिलता या लापरवाही को अत्यन्त गम्भीरता से लेकर सम्बन्धित का उत्तरदायित्व निर्धारण करते हुए कार्यवाही की जाये।

(देवेन्द्र सिंह चौहान)

समस्त पुलिस आयुक्त, उत्तर प्रदेश

समस्त पुलिस उपमहानिरीक्षक / वरिष्ठ पुलिस अधीक्षक / पुलिस अधीक्षक, प्रभारी जनपद उत्तर प्रदेश

प्रतिलिपि- निम्नांकित अधिकारियों को सूचनार्थ एवं आवश्यक कार्यवाही हेतु प्रेषित:

1. समस्त पुलिस महानिदेशक, उ.प्र.
2. समस्त जोनल अपर पुलिस महानिदेशक, उ.प्र.
3. समस्त अपर पुलिस महानिदेशक, उ.प्र.
4. समस्त पुलिस महानिरीक्षक/उपमहानिरीक्षक परिक्षेत्र, उ.प्र.
"""

output10 = """
देवेन्द्र सिंह चौहान,

डीजी परिपत्र सं0 - 09 /2023

आई0पी0एस0 पुलिस महानिदेशक

उत्तर प्रदेश

पुलिस मुख्यालय, गोमती नगर विस्तार,

उत्तर प्रदेश

लखनऊ-226010

दिनांक: मार्च 02, 2023

विषय: रिट सी. संख्या 29605/2022 बासु यादव बनाम भारत संघ व 4 अन्य में मा0 उच्च न्यायालय इलाहाबाद द्वारा पारित आदेश दिनांकित 16.12.2022 के अनुपालन में पासपोर्ट आवेदन पत्र के पुलिस सत्यापन हेतु दिशा-निर्देश

प्रिय महोदया / महोदय,

पासपोर्ट (Passport) जारी करने हेतु प्रस्तुत किये गये आवेदन पत्रों के संदर्भ में आवेदकों का पुलिस सत्यापन, सम्बन्धित थाना द्वारा किया जाता है। आवेदक बासु यादव निवासी जनपद आजमगढ़ द्वारा प्रस्तुत पासपोर्ट आवेदन पत्र पर स्थानीय पुलिस द्वारा पासपोर्ट न प्रदान किये जाने की संस्तुति के विरुद्ध योजित सिविल मिस. रिट (सी) संख्या: 29605/2022 बासु यादव बनाम उ0प्र0 राज्य में मा0 उच्च न्यायालय द्वारा पारित आदेश दिनांकित 16.12.2022 में पासपोर्ट आवेदन पत्र सत्यापन के सम्बन्ध में निम्नवत निर्देश दिये गये हैं:

Having heard learned counsel for the petitioner and learned Standing Counsel and having gone through the instructions which have been sent by the Director General of Police, the Court is definitely of the view that no non-cognizable report which was registered could be taken into cognizance if no investigation was ordered by the concerned Magistrate. Even though in the instant case, whether the passport can be refused on the basis of the pendency of the criminal case is not the question involved, we are of the view that even during the pendency of any criminal case, passport could be issued/renewed as per the Government Order dated 25.8.1993 if the Court passes orders for that purpose. In the instant case, we find that the application of the petitioner was rejected on the basis of the two reports of non-cognizable cases, namely NCR No. 112/2012 and NCR No. 118/2018. The Director General of Police has also given his view that the reports with regard to the non-cognizable cases could not be made the basis for rejecting an application for issuance of a passport if they had not been investigated into. Under such circumstances, we issue the following directions:

(1) The passport form of the petitioner for the issuance of a passport be considered within a period of two weeks from the date of presentation of a certified copy of this order before the respondent no. 2 Regional Passport Officer, Regional Passport Office, Vipin Khand, Gomti Nagar.

(2) Since we are finding that in quite a few cases the reports of non-cognizable cases in which the concerned Magistrate had not ordered for investigation were being taken into account for rejection of passport, we issue a direction to the Director General of Police to instruct his officers to give the report with regard to the pendency of reports in non-cognizable cases after appropriate and proper application of mind.

(3) Outright the passport applications be not rejected under section 6(2)(f) of the Passports Act if orders of the Court where the criminal case is pending, have been passed as per the Government Order dated 25.8.1993. The Director General of Police to issue notification in this regard also.

उल्लेखनीय है कि विदेश मंत्रालय, भारत सरकार द्वारा अपने पत्र संख्या: PMU Dir (PV PSP)/133/11 दिनांक 21.05.2018, पत्र संख्या: V/l/401/1/4/2013 दिनांक 26.10.2018 तथा पत्र संख्या: VI/401/01/04/2013Pt. दिनांक 22.08.2019 (छायाप्रतियाँ संलग्न) के माध्यम से प्रेषित Police verification Report Proforma Process के अनुसार पासपोर्ट सत्यापन की कार्यवाही एम.पासपोर्ट ऐप के माध्यम से प्रचलित है। वर्तमान व्यवस्था के अन्तर्गत उक्त Police verification Report Proforma में जनपद स्तर से होने वाले सत्यापन का बिन्दु (पार्ट-ए) नागरिकता व आपराधिक क्रियाकलाप के सम्बन्ध में थाना स्तर से ही सत्यापन की कार्यवाही की जा रही है। विदेश मंत्रालय भारत सरकार के कार्यालय ज्ञाप संख्या: I/401/1/5/2019 दिनांकित 10.10.2019 (छायाप्रति संलग्न) द्वारा ऐसे आवेदकों, जिनके विरुद्ध आपराधिक मामले प्रचलित हैं, को पासपोर्ट जारी किये जाने के सम्बन्ध में दिशा-निर्देश निर्गत किये गये हैं।

उपरोक्त संदर्भ में आप सभी को निर्देशित किया जाता है कि मा0 उच्च न्यायालय द्वारा पारित आदेश दिनांकित 16.12.2022 तथा पासपोर्ट निर्गत किये जाने की प्रक्रिया के सम्बन्ध में विदेश मंत्रालय भारत सरकार द्वारा जारी निर्देशों का भलीभाँति अध्ययन करते हुये अपने अधीनस्थों को भी अवगत करायें तथा पासपोर्ट आवेदन पत्रों का सत्यापन करते समय इन निर्देशों का कड़ाई से अनुपालन सुनिश्चित किया जाए। पासपोर्ट आवेदन पत्रों के सत्यापन में पूर्ण सतर्कता बरती जाए तथा आवेदक के सम्बन्ध में थाने के रिकॉर्ड में उपलब्ध समस्त सूचनाओं के आधार पर ही संस्तुति अंकित की जाए। यह भी सुनिश्चित किया जाए कि बिना उचित एवं पर्याप्त कारण के यंत्रवत किसी आवेदक के पासपोर्ट आवेदन पत्र को निरस्त करने की संस्तुति न की जाए।

(देवेन्द्र सिंह चौहान)

1. पुलिस आयुक्त,
कमिश्नरेट लखनऊ, कानपुर, वाराणसी, गौतमबुद्धनगर, आगरा, गाजियाबाद, प्रयागराज

2. समस्त वरिष्ठ पुलिस अधीक्षक/पुलिस अधीक्षक, प्रभारी जनपद, रेलवेज, उत्तर प्रदेश

प्रतिलिपि: निम्नलिखित को सूचनार्थ एवं आवश्यक कार्यवाही हेतु:

1. समस्त पुलिस महानिदेशक, उ0प्र0
2. समस्त जोनल अपर पुलिस महानिदेशक, उ0प्र0
3. समस्त अपर पुलिस महानिदेशक, उ0प्र0
4. समस्त पुलिस महानिरीक्षक/उपमहानिरीक्षक परिक्षेत्र, उ0प्र0
"""



In [6]:
outputs_gpt = []
outputs_gpt.append(output1)
outputs_gpt.append(output2)
outputs_gpt.append(output3)
outputs_gpt.append(output4)
outputs_gpt.append(output5)
outputs_gpt.append(output6)
outputs_gpt.append(output7)
outputs_gpt.append(output8)
outputs_gpt.append(output9)
outputs_gpt.append(output10)

In [7]:
print(outputs_gpt[0])


देवेन्द्र सिंह चौहान, डीजी परिपत्र संख्या- 01/ 2023

आईपीएस पुलिस महानिदेशक,

उत्तर प्रदेश

पुलिस मुख्यालय, लखनऊ

दिनांक: लखनऊ: जनवरी 03, 2023

विषय: शरद ऋतु में शीतलहर/घने कोहरे की रात्रि में घुमक्कड़ बावरियों/कच्छा बनियानधारी गिरोहों के अपराधियों पर प्रभावी अंकुश लगाये जाने तथा अपराध नियन्त्रण हेतु आवश्यक दिशा-निर्देश

प्रिय महोदय/महोदया,

आप अवगत हैं कि शरद ऋतु में शीतलहर घने कोहरे की रात्रि में विशेष रूप से सड़कों पर आम जनमानस का आवागमन कम संख्या में होता है। इस मौसम में घुमक्कड़ अपराधी बावरियां/कच्छा बनियान गैंग द्वारा सनसनीखेज अपराध कारित करने की सम्भावना रहती है। ऐसी घटनाएं प्रायः नगरों तथा कस्बों व कभी-कभी ग्रामीण अंचल के बाहरी क्षेत्रों में घटित होती हैं।

डीजी परिपत्र संख्या-01/2021, दिनांक 01.01.2021, आप सहमत होंगे कि इस प्रकार की घटना डीजी-परिपत्र संख्या-25/2011, दिनांक 27.06.2019 घटित होने पर जहां एक ओर जनमानस में डीजी परिपत्र संख्या-67/2018, दिनांक 23.11.2018 असुरक्षा का भय व्याप्त होता है, वहीं दूसरी ओर डीजी-परिपत्र संख्या-47/2016, दिनांक 03.08.2016 पुलिस के प्रति आक्रो

In [8]:
print(outputs_gpt[1])


डीजी परिपत्र संख्या: 01/2023

देवेन्द्र सिंह चौहान

पुलिस महानिदेशक,

आई.पी.एस.

उत्तर प्रदेश

पुलिस मुख्यालय गोमती नगर विस्तार,

लखनऊ-226010

दिनांक: जनवरी 27, 2023

विषय: मा. संसद सदस्य एवं राज्य विधान मण्डल के प्रति शिष्टाचार / अनुमन्य प्रोटोकॉल एवं सौजन्य प्रदर्शन के अनुपालन के संबंध में मा. सदस्यों एवं अन्य जन प्रतिनिधियों के

प्रिय महोदय / महोदया,

आपका ध्यान मा. संसद सदस्य एवं राज्य विधान मण्डल के मा. सदस्यों एवं जनप्रतिनिधियों के प्रति शिष्टाचार / अनुमन्य प्रोटोकॉल एवं सौजन्य प्रदर्शन हेतु प्रचलित व्यवस्था की ओर आकृष्ट करना चाहूंगा। मा. संसद सदस्य एवं राज्य विधान मण्डल के मा. सदस्यों के प्रति शिष्टाचार / अनुमन्य प्रोटोकॉल के संबंध में उ.प्र. शासन एवं इस मुख्यालय से समय-समय पर पार्श्वांकित कॉलम में अंकित शासनादेश एवं निर्देश निर्गत किये गये हैं। यह संज्ञान में आया है कि इन निर्देशों के सम्यक अनुपालन में शिथिलता बरती जा रही है।

उक्त संबंध में समय-समय पर निर्गत आदेशों एवं निर्देशों का प्रत्येक थाना, सर्किल, जनपद, परिक्षेत्र, जोन एवं पुलिस कमिश्नरेट स्तर पर अनुपालन करते हुए गुणवत

In [9]:
print(outputs_gpt[2])


देवेन्द्र सिंह चौहान,

डीजी परिपत्र सं0 - 05/2023

पुलिस महानिदेशक,

उत्तर प्रदेश

पुलिस मुख्यालय, गोमती नगर विस्तार,

लखनऊ-226010

दिनांक: जनवरी 30, 2023

विषय: मा. उच्च न्यायालय इलाहाबाद तथा खण्डपीठ लखनऊ में योजित पत्र याचिकाओं (Letter Petitions) पर तथ्यपरक, प्रासंगिक एवं विधिक आख्या प्रेषित किये जाने के सम्बन्ध में।

प्रिय महोदया / महोदय,

अवगत कराना है कि वर्तमान में मा. उच्च न्यायालय इलाहाबाद तथा खण्डपीठ लखनऊ में पत्र याचिकाएं (Letter Petitions) काफी संख्या में प्रस्तुत हो रही हैं। सरकार की ओर से प्रतिवाद करने वाले विधि अधिकारियों द्वारा ऐसी पत्र याचिकाओं पर अल्प समय में आख्या उपलब्ध कराने की अपेक्षा की जाती है। जिस कारण ऐसी आख्या सामान्यतया थानाध्यक्ष, चौकी प्रभारी, हल्का प्रभारी एवं अन्य उप निरीक्षक द्वारा सीधे प्रेषित की जा रही हैं।

मा. उच्च न्यायालय में सरकार की ओर से प्रतिवाद करने वाले विधि अधिकारियों द्वारा यह अवगत कराया गया है कि पत्र याचिकाओं (Letter Petitions) पर प्रेषित आख्या (Comments) में इस प्रकार के तथ्यों का समावेश किया जा रहा है, जो न तो विधि सम्मत होते हैं और न 

In [12]:
count = 0
for doc in outputs_gpt:
    text_file_path = f"{folder_list[count]}.txt"
    count += 1
    with open(text_file_path, "w", encoding="utf-8") as text_file:
        text_file.write(doc)